In [4]:
path = "C:/Users/RAZER/Desktop/TEMp/New folder/KINECT_dataset_with_qor15.csv"

In [1]:
path = r"D:\Data\NYC\KINZ\KINECT_dataset_with_qor15.csv"

In [7]:
file_path = r"D:\Data\NYC\KINZ\KINECT_dataset_with_qor15.csv"

In [2]:
import torch
import pandas as pd
import plotly.graph_objs as go
from torch_geometric.data import Data

# Load the dataset

df = pd.read_csv(file_path)

# Define anatomical connections as edges
edges = torch.tensor([
    [0, 2],  # FOOT_RIGHT to ANKLE_RIGHT
    [1, 3],  # FOOT_LEFT to ANKLE_LEFT
    [2, 4],  # ANKLE_RIGHT to KNEE_RIGHT
    [3, 5],  # ANKLE_LEFT to KNEE_LEFT
    [4, 6],  # KNEE_RIGHT to HIP_RIGHT
    [5, 7],  # KNEE_LEFT to HIP_LEFT
    [6, 8],  # HIP_RIGHT to PELVIS
    [7, 8],  # HIP_LEFT to PELVIS
    [8, 9],  # PELVIS to SPINE_NAVAL
    [9, 10], # SPINE_NAVAL to SPINE_CHEST
    [10, 11],# SPINE_CHEST to CLAVICLE_RIGHT
    [10, 12],# SPINE_CHEST to CLAVICLE_LEFT
    [11, 13],# CLAVICLE_RIGHT to SHOULDER_RIGHT
    [12, 14],# CLAVICLE_LEFT to SHOULDER_LEFT
    [13, 15],# SHOULDER_RIGHT to ELBOW_RIGHT
    [14, 16],# SHOULDER_LEFT to ELBOW_LEFT
    [15, 17],# ELBOW_RIGHT to WRIST_RIGHT
    [16, 18],# ELBOW_LEFT to WRIST_LEFT
    [17, 19],# WRIST_RIGHT to HAND_RIGHT
    [18, 20],# WRIST_LEFT to HAND_LEFT
    [19, 21],# HAND_RIGHT to HANDTIP_RIGHT
    [20, 22],# HAND_LEFT to HANDTIP_LEFT
    [21, 23],# HANDTIP_RIGHT to THUMB_RIGHT
    [22, 24],# HANDTIP_LEFT to THUMB_LEFT
    [10, 25],# SPINE_CHEST to NECK
    [25, 26],# NECK to HEAD
]).t().contiguous()

# List of joints in the order they appear in the DataFrame
joints = [
    'FOOT_RIGHT', 'FOOT_LEFT', 'ANKLE_RIGHT', 'ANKLE_LEFT', 'KNEE_RIGHT', 'KNEE_LEFT',
    'HIP_RIGHT', 'HIP_LEFT', 'PELVIS', 'SPINE_NAVAL', 'SPINE_CHEST',
    'CLAVICLE_RIGHT', 'CLAVICLE_LEFT', 'SHOULDER_RIGHT', 'SHOULDER_LEFT',
    'ELBOW_RIGHT', 'ELBOW_LEFT', 'WRIST_RIGHT', 'WRIST_LEFT', 'HAND_RIGHT',
    'HAND_LEFT', 'HANDTIP_RIGHT', 'HANDTIP_LEFT', 'THUMB_RIGHT', 'THUMB_LEFT',
    'NECK', 'HEAD', 'NOSE', 'EYE_LEFT', 'EAR_LEFT', 'EYE_RIGHT', 'EAR_RIGHT'
]

# Function to extract node features and create a Data object
def create_data_object_vis(row):
    node_features = []
    node_positions = []  # Store (x, y, z) positions separately for Plotly visualization

    for joint in joints:
        x = row[f'{joint}_X']
        y = row[f'{joint}_Y']
        z = row[f'{joint}_Z']
        node_features.append([x, y, z])
        node_positions.append((x, y, z))  # Save position for each joint

    node_features = torch.tensor(node_features, dtype=torch.float)

    # Extract label (assuming 'frailty' is the target column)
    label = torch.tensor([row['QoR_class']], dtype=torch.float)

    # Create the Data object
    data = Data(x=node_features, edge_index=edges, y=label)

    return data, node_positions

# Function to plot the graph using Plotly
def plot_graph(node_positions):
    # Extract x, y, z coordinates from node_positions
    x_coords = [pos[0] for pos in node_positions]
    y_coords = [pos[1] for pos in node_positions]
    z_coords = [pos[2] for pos in node_positions]

    # Create edges for Plotly
    edge_x = []
    edge_y = []
    edge_z = []
    for edge in edges.t().tolist():
        x0, y0, z0 = node_positions[edge[0]]
        x1, y1, z1 = node_positions[edge[1]]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])
        edge_z.extend([z0, z1, None])

    # Plot the edges
    edge_trace = go.Scatter3d(
        x=edge_x, y=edge_y, z=edge_z,
        mode='lines',
        line=dict(color='black', width=2),
        hoverinfo='none'
    )

    # Plot the nodes with joint names
    node_trace = go.Scatter3d(
        x=x_coords, y=y_coords, z=z_coords,
        mode='markers+text',
        marker=dict(size=6, color='blue'),
        text=joints,  # Use the joint names as text labels
        hoverinfo='text'
    )

    # Create the figure with gridlines and joint names
    fig = go.Figure(data=[edge_trace, node_trace],
                    layout=go.Layout(
                        title='3D  Visualization',
                        showlegend=False,
                        scene=dict(
                            xaxis=dict(
                                showbackground=True,
                                backgroundcolor="rgb(230, 230, 230)",
                                gridcolor="rgb(200, 200, 200)",
                                showgrid=True,
                                zerolinecolor="rgb(200, 200, 200)",
                            ),
                            yaxis=dict(
                                showbackground=True,
                                backgroundcolor="rgb(230, 230, 230)",
                                gridcolor="rgb(200, 200, 200)",
                                showgrid=True,
                                zerolinecolor="rgb(200, 200, 200)",
                            ),
                            zaxis=dict(
                                showbackground=True,
                                backgroundcolor="rgb(230, 230, 230)",
                                gridcolor="rgb(200, 200, 200)",
                                showgrid=True,
                                zerolinecolor="rgb(200, 200, 200)",
                            ),
                        ),
                        margin=dict(l=0, r=0, b=0, t=40),
                    ))

    fig.show()

# Example usage with a single row (e.g., the first row in your DataFrame)
single_row = df.iloc[0]
data, node_positions = create_data_object_vis(single_row)

# Plot the graph using Plotly
plot_graph(node_positions)


In [4]:
import torch
import pandas as pd
import plotly.graph_objs as go
from torch_geometric.data import Data

# Load the dataset

df = pd.read_csv(file_path)

# Define anatomical connections as edges
edges = torch.tensor([
    [0, 2],  # FOOT_RIGHT to ANKLE_RIGHT
    [1, 3],  # FOOT_LEFT to ANKLE_LEFT
    [2, 4],  # ANKLE_RIGHT to KNEE_RIGHT
    [3, 5],  # ANKLE_LEFT to KNEE_LEFT
    [4, 6],  # KNEE_RIGHT to HIP_RIGHT
    [5, 7],  # KNEE_LEFT to HIP_LEFT
    [6, 8],  # HIP_RIGHT to PELVIS
    [7, 8],  # HIP_LEFT to PELVIS
    [8, 9],  # PELVIS to SPINE_NAVAL
    [9, 10], # SPINE_NAVAL to SPINE_CHEST
    [10, 11],# SPINE_CHEST to CLAVICLE_RIGHT
    [10, 12],# SPINE_CHEST to CLAVICLE_LEFT
    [11, 13],# CLAVICLE_RIGHT to SHOULDER_RIGHT
    [12, 14],# CLAVICLE_LEFT to SHOULDER_LEFT
    [13, 15],# SHOULDER_RIGHT to ELBOW_RIGHT
    [14, 16],# SHOULDER_LEFT to ELBOW_LEFT
    [15, 17],# ELBOW_RIGHT to WRIST_RIGHT
    [16, 18],# ELBOW_LEFT to WRIST_LEFT
    [17, 19],# WRIST_RIGHT to HAND_RIGHT
    [18, 20],# WRIST_LEFT to HAND_LEFT
    [19, 21],# HAND_RIGHT to HANDTIP_RIGHT
    [20, 22],# HAND_LEFT to HANDTIP_LEFT
    [21, 23],# HANDTIP_RIGHT to THUMB_RIGHT
    [22, 24],# HANDTIP_LEFT to THUMB_LEFT
    [10, 25],# SPINE_CHEST to NECK
    [25, 26],# NECK to HEAD
]).t().contiguous()

# List of joints in the order they appear in the DataFrame
joints = [
    'FOOT_RIGHT', 'FOOT_LEFT', 'ANKLE_RIGHT', 'ANKLE_LEFT', 'KNEE_RIGHT', 'KNEE_LEFT',
    'HIP_RIGHT', 'HIP_LEFT', 'PELVIS', 'SPINE_NAVAL', 'SPINE_CHEST',
    'CLAVICLE_RIGHT', 'CLAVICLE_LEFT', 'SHOULDER_RIGHT', 'SHOULDER_LEFT',
    'ELBOW_RIGHT', 'ELBOW_LEFT', 'WRIST_RIGHT', 'WRIST_LEFT', 'HAND_RIGHT',
    'HAND_LEFT', 'HANDTIP_RIGHT', 'HANDTIP_LEFT', 'THUMB_RIGHT', 'THUMB_LEFT',
    'NECK', 'HEAD', 'NOSE', 'EYE_LEFT', 'EAR_LEFT', 'EYE_RIGHT', 'EAR_RIGHT'
]

# Function to extract node features and create a Data object
def create_data_object_vis(row):
    node_features = []
    node_positions = []  # Store (x, y, z) positions separately for Plotly visualization

    for joint in joints:
        x = row[f'{joint}_X']
        y = row[f'{joint}_Y']
        z = row[f'{joint}_Z']
        node_features.append([x, y, z])
        node_positions.append((x, y, z))  # Save position for each joint

    node_features = torch.tensor(node_features, dtype=torch.float)

    # Extract label (assuming 'frailty' is the target column)
    label = torch.tensor([row['QoR_class']], dtype=torch.float)

    # Create the Data object
    data = Data(x=node_features, edge_index=edges, y=label)

    return data, node_positions

# Function to plot the graph using Plotly
def plot_graph(node_positions):
    # Extract x, y, z coordinates from node_positions
    x_coords = [pos[0] for pos in node_positions]
    y_coords = [pos[1] for pos in node_positions]
    z_coords = [pos[2] for pos in node_positions]

    # Create edges for Plotly
    edge_x = []
    edge_y = []
    edge_z = []
    for edge in edges.t().tolist():
        x0, y0, z0 = node_positions[edge[0]]
        x1, y1, z1 = node_positions[edge[1]]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])
        edge_z.extend([z0, z1, None])

    # Plot the edges
    edge_trace = go.Scatter3d(
        x=edge_x, y=edge_y, z=edge_z,
        mode='lines',
        line=dict(color='black', width=2),
        hoverinfo='none'
    )

    # Plot the nodes with joint names and coordinates
    node_trace = go.Scatter3d(
        x=x_coords, y=y_coords, z=z_coords,
        mode='markers+text',
        marker=dict(size=6, color='blue'),
        text=[f'{joint}<br>({x:.2f}, {y:.2f}, {z:.2f})' for joint, x, y, z in zip(joints, x_coords, y_coords, z_coords)],  # Joint names and coordinates
        hoverinfo='text'
    )

    # Create the figure with gridlines and joint names
    fig = go.Figure(data=[edge_trace, node_trace],
                    layout=go.Layout(
                        title='3D Joint Visualization',
                        showlegend=False,
                        scene=dict(
                            xaxis=dict(
                                showbackground=True,
                                backgroundcolor="rgb(230, 230, 230)",
                                gridcolor="rgb(200, 200, 200)",
                                showgrid=True,
                                zerolinecolor="rgb(200, 200, 200)",
                            ),
                            yaxis=dict(
                                showbackground=True,
                                backgroundcolor="rgb(230, 230, 230)",
                                gridcolor="rgb(200, 200, 200)",
                                showgrid=True,
                                zerolinecolor="rgb(200, 200, 200)",
                            ),
                            zaxis=dict(
                                showbackground=True,
                                backgroundcolor="rgb(230, 230, 230)",
                                gridcolor="rgb(200, 200, 200)",
                                showgrid=True,
                                zerolinecolor="rgb(200, 200, 200)",
                            ),
                        ),
                        margin=dict(l=0, r=0, b=0, t=40),
                    ))

    fig.show()

# Example usage with a single row (e.g., the first row in your DataFrame)
single_row = df.iloc[0]
data, node_positions = create_data_object_vis(single_row)

# Plot the graph using Plotly
plot_graph(node_positions)


In [8]:
import torch
import pandas as pd
import plotly.graph_objs as go
from torch_geometric.data import Data

# Define anatomical connections as edges
edges = torch.tensor([
    [0, 2],  # FOOT_RIGHT to ANKLE_RIGHT
    [1, 3],  # FOOT_LEFT to ANKLE_LEFT
    [2, 4],  # ANKLE_RIGHT to KNEE_RIGHT
    [3, 5],  # ANKLE_LEFT to KNEE_LEFT
    [4, 6],  # KNEE_RIGHT to HIP_RIGHT
    [5, 7],  # KNEE_LEFT to HIP_LEFT
    [6, 8],  # HIP_RIGHT to PELVIS
    [7, 8],  # HIP_LEFT to PELVIS
    [8, 9],  # PELVIS to SPINE_NAVAL
    [9, 10], # SPINE_NAVAL to SPINE_CHEST
    [10, 11],# SPINE_CHEST to CLAVICLE_RIGHT
    [10, 12],# SPINE_CHEST to CLAVICLE_LEFT
    [11, 13],# CLAVICLE_RIGHT to SHOULDER_RIGHT
    [12, 14],# CLAVICLE_LEFT to SHOULDER_LEFT
    [13, 15],# SHOULDER_RIGHT to ELBOW_RIGHT
    [14, 16],# SHOULDER_LEFT to ELBOW_LEFT
    [15, 17],# ELBOW_RIGHT to WRIST_RIGHT
    [16, 18],# ELBOW_LEFT to WRIST_LEFT
    [17, 19],# WRIST_RIGHT to HAND_RIGHT
    [18, 20],# WRIST_LEFT to HAND_LEFT
    [19, 21],# HAND_RIGHT to HANDTIP_RIGHT
    [20, 22],# HAND_LEFT to HANDTIP_LEFT
    [21, 23],# HANDTIP_RIGHT to THUMB_RIGHT
    [22, 24],# HANDTIP_LEFT to THUMB_LEFT
    [10, 25],# SPINE_CHEST to NECK
    [25, 26],# NECK to HEAD
]).t().contiguous()

# List of joints in the order they appear
joints = [
    'FOOT_RIGHT', 'FOOT_LEFT', 'ANKLE_RIGHT', 'ANKLE_LEFT', 'KNEE_RIGHT', 'KNEE_LEFT',
    'HIP_RIGHT', 'HIP_LEFT', 'PELVIS', 'SPINE_NAVAL', 'SPINE_CHEST',
    'CLAVICLE_RIGHT', 'CLAVICLE_LEFT', 'SHOULDER_RIGHT', 'SHOULDER_LEFT',
    'ELBOW_RIGHT', 'ELBOW_LEFT', 'WRIST_RIGHT', 'WRIST_LEFT', 'HAND_RIGHT',
    'HAND_LEFT', 'HANDTIP_RIGHT', 'HANDTIP_LEFT', 'THUMB_RIGHT', 'THUMB_LEFT',
    'NECK', 'HEAD'
]

# Create mock data for a humanoid figure
def create_mock_data():
    mock_data = {
        'FOOT_RIGHT_X': 0, 'FOOT_RIGHT_Y': 0, 'FOOT_RIGHT_Z': 0,
        'FOOT_LEFT_X': 1, 'FOOT_LEFT_Y': 0, 'FOOT_LEFT_Z': 0,
        'ANKLE_RIGHT_X': 0, 'ANKLE_RIGHT_Y': 1, 'ANKLE_RIGHT_Z': 0,
        'ANKLE_LEFT_X': 1, 'ANKLE_LEFT_Y': 1, 'ANKLE_LEFT_Z': 0,
        'KNEE_RIGHT_X': 0, 'KNEE_RIGHT_Y': 2, 'KNEE_RIGHT_Z': 0,
        'KNEE_LEFT_X': 1, 'KNEE_LEFT_Y': 2, 'KNEE_LEFT_Z': 0,
        'HIP_RIGHT_X': 0, 'HIP_RIGHT_Y': 3, 'HIP_RIGHT_Z': 0,
        'HIP_LEFT_X': 1, 'HIP_LEFT_Y': 3, 'HIP_LEFT_Z': 0,
        'PELVIS_X': 0.5, 'PELVIS_Y': 3.5, 'PELVIS_Z': 0,
        'SPINE_NAVAL_X': 0.5, 'SPINE_NAVAL_Y': 4, 'SPINE_NAVAL_Z': 0,
        'SPINE_CHEST_X': 0.5, 'SPINE_CHEST_Y': 5, 'SPINE_CHEST_Z': 0,
        'CLAVICLE_RIGHT_X': 0, 'CLAVICLE_RIGHT_Y': 5.5, 'CLAVICLE_RIGHT_Z': 0,
        'CLAVICLE_LEFT_X': 1, 'CLAVICLE_LEFT_Y': 5.5, 'CLAVICLE_LEFT_Z': 0,
        'SHOULDER_RIGHT_X': 0, 'SHOULDER_RIGHT_Y': 6, 'SHOULDER_RIGHT_Z': 0,
        'SHOULDER_LEFT_X': 1, 'SHOULDER_LEFT_Y': 6, 'SHOULDER_LEFT_Z': 0,
        'ELBOW_RIGHT_X': 0, 'ELBOW_RIGHT_Y': 7, 'ELBOW_RIGHT_Z': 0,
        'ELBOW_LEFT_X': 1, 'ELBOW_LEFT_Y': 7, 'ELBOW_LEFT_Z': 0,
        'WRIST_RIGHT_X': 0, 'WRIST_RIGHT_Y': 8, 'WRIST_RIGHT_Z': 0,
        'WRIST_LEFT_X': 1, 'WRIST_LEFT_Y': 8, 'WRIST_LEFT_Z': 0,
        'HAND_RIGHT_X': 0, 'HAND_RIGHT_Y': 9, 'HAND_RIGHT_Z': 0,
        'HAND_LEFT_X': 1, 'HAND_LEFT_Y': 9, 'HAND_LEFT_Z': 0,
        'HANDTIP_RIGHT_X': 0, 'HANDTIP_RIGHT_Y': 9.5, 'HANDTIP_RIGHT_Z': 0,
        'HANDTIP_LEFT_X': 1, 'HANDTIP_LEFT_Y': 9.5, 'HANDTIP_LEFT_Z': 0,
        'THUMB_RIGHT_X': -0.5, 'THUMB_RIGHT_Y': 9, 'THUMB_RIGHT_Z': 0,
        'THUMB_LEFT_X': 1.5, 'THUMB_LEFT_Y': 9, 'THUMB_LEFT_Z': 0,
        'NECK_X': 0.5, 'NECK_Y': 6.5, 'NECK_Z': 0,
        'HEAD_X': 0.5, 'HEAD_Y': 7, 'HEAD_Z': 0,
    }

    return pd.Series(mock_data)

# Function to extract node features and create a Data object
def create_data_object_vis(row):
    node_features = []
    node_positions = []  # Store (x, y, z) positions separately for Plotly visualization

    for joint in joints:
        x = row[f'{joint}_X']
        y = row[f'{joint}_Y']
        z = row[f'{joint}_Z']
        node_features.append([x, y, z])
        node_positions.append((x, y, z))  # Save position for each joint

    node_features = torch.tensor(node_features, dtype=torch.float)

    # No label in this mock data
    label = torch.tensor([0], dtype=torch.float)

    # Create the Data object
    data = Data(x=node_features, edge_index=edges, y=label)

    return data, node_positions

# Function to plot the graph using Plotly
def plot_graph_large(node_positions):
    # Extract x, y, z coordinates from node_positions
    x_coords = [pos[0] for pos in node_positions]
    y_coords = [pos[1] for pos in node_positions]
    z_coords = [pos[2] for pos in node_positions]

    # Create edges for Plotly
    edge_x = []
    edge_y = []
    edge_z = []
    for edge in edges.t().tolist():
        x0, y0, z0 = node_positions[edge[0]]
        x1, y1, z1 = node_positions[edge[1]]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])
        edge_z.extend([z0, z1, None])

    # Plot the edges
    edge_trace = go.Scatter3d(
        x=edge_x, y=edge_y, z=edge_z,
        mode='lines',
        line=dict(color='black', width=3),  # Increased width for better visibility
        hoverinfo='none'
    )

    # Plot the nodes with joint names
    node_trace = go.Scatter3d(
        x=x_coords, y=y_coords, z=z_coords,
        mode='markers+text',
        marker=dict(size=8, color='blue'),  # Increased size for better visibility
        text=joints,  # Use the joint names as text labels
        hoverinfo='text'
    )

    # Create the figure with increased size
    fig = go.Figure(data=[edge_trace, node_trace],
                    layout=go.Layout(
                        title='3D Humanoid Skeleton Visualization',
                        showlegend=False,
                        scene=dict(
                            xaxis=dict(
                                showbackground=True,
                                backgroundcolor="rgb(230, 230, 230)",
                                gridcolor="rgb(200, 200, 200)",
                                showgrid=True,
                                zerolinecolor="rgb(200, 200, 200)",
                            ),
                            yaxis=dict(
                                showbackground=True,
                                backgroundcolor="rgb(230, 230, 230)",
                                gridcolor="rgb(200, 200, 200)",
                                showgrid=True,
                                zerolinecolor="rgb(200, 200, 200)",
                            ),
                            zaxis=dict(
                                showbackground=True,
                                backgroundcolor="rgb(230, 230, 230)",
                                gridcolor="rgb(200, 200, 200)",
                                showgrid=True,
                                zerolinecolor="rgb(200, 200, 200)",
                            ),
                        ),
                        width=1200,  # Increased width
                        height=800,  # Increased height
                        margin=dict(l=0, r=0, b=0, t=40),
                    ))

    fig.show()

# Create mock data for a humanoid figure
mock_data = create_mock_data()

# Convert the mock data to a Data object and node positions
data, node_positions = create_data_object_vis(mock_data)

# Plot the larger graph using Plotly
plot_graph_large(node_positions)


In [10]:
import torch
import pandas as pd
import plotly.graph_objs as go
from torch_geometric.data import Data

# Load the dataset

df = pd.read_csv(file_path)

# Define anatomical connections as edges
edges = torch.tensor([
    [0, 2],  # FOOT_RIGHT to ANKLE_RIGHT
    [1, 3],  # FOOT_LEFT to ANKLE_LEFT
    [2, 4],  # ANKLE_RIGHT to KNEE_RIGHT
    [3, 5],  # ANKLE_LEFT to KNEE_LEFT
    [4, 6],  # KNEE_RIGHT to HIP_RIGHT
    [5, 7],  # KNEE_LEFT to HIP_LEFT
    [6, 8],  # HIP_RIGHT to PELVIS
    [7, 8],  # HIP_LEFT to PELVIS
    [8, 9],  # PELVIS to SPINE_NAVAL
    [9, 10], # SPINE_NAVAL to SPINE_CHEST
    [10, 11],# SPINE_CHEST to CLAVICLE_RIGHT
    [10, 12],# SPINE_CHEST to CLAVICLE_LEFT
    [11, 13],# CLAVICLE_RIGHT to SHOULDER_RIGHT
    [12, 14],# CLAVICLE_LEFT to SHOULDER_LEFT
    [13, 15],# SHOULDER_RIGHT to ELBOW_RIGHT
    [14, 16],# SHOULDER_LEFT to ELBOW_LEFT
    [15, 17],# ELBOW_RIGHT to WRIST_RIGHT
    [16, 18],# ELBOW_LEFT to WRIST_LEFT
    [17, 19],# WRIST_RIGHT to HAND_RIGHT
    [18, 20],# WRIST_LEFT to HAND_LEFT
    [19, 21],# HAND_RIGHT to HANDTIP_RIGHT
    [20, 22],# HAND_LEFT to HANDTIP_LEFT
    [21, 23],# HANDTIP_RIGHT to THUMB_RIGHT
    [22, 24],# HANDTIP_LEFT to THUMB_LEFT
    [10, 25],# SPINE_CHEST to NECK
    [25, 26],# NECK to HEAD
]).t().contiguous()

# List of joints in the order they appear in the DataFrame
joints = [
    'FOOT_RIGHT', 'FOOT_LEFT', 'ANKLE_RIGHT', 'ANKLE_LEFT', 'KNEE_RIGHT', 'KNEE_LEFT',
    'HIP_RIGHT', 'HIP_LEFT', 'PELVIS', 'SPINE_NAVAL', 'SPINE_CHEST',
    'CLAVICLE_RIGHT', 'CLAVICLE_LEFT', 'SHOULDER_RIGHT', 'SHOULDER_LEFT',
    'ELBOW_RIGHT', 'ELBOW_LEFT', 'WRIST_RIGHT', 'WRIST_LEFT', 'HAND_RIGHT',
    'HAND_LEFT', 'HANDTIP_RIGHT', 'HANDTIP_LEFT', 'THUMB_RIGHT', 'THUMB_LEFT',
    'NECK', 'HEAD', 'NOSE', 'EYE_LEFT', 'EAR_LEFT', 'EYE_RIGHT', 'EAR_RIGHT'
]

# Function to extract node features and create a Data object
def create_data_object_vis(row):
    node_features = []
    node_positions = []  # Store (x, y, z) positions separately for Plotly visualization

    for joint in joints:
        x = row[f'{joint}_X']
        y = row[f'{joint}_Y']
        z = row[f'{joint}_Z']
        node_features.append([x, y, z])
        node_positions.append((x, y, z))  # Save position for each joint

    node_features = torch.tensor(node_features, dtype=torch.float)

    # Extract label (assuming 'frailty' is the target column)
    label = torch.tensor([row['QoR_class']], dtype=torch.float)

    # Create the Data object
    data = Data(x=node_features, edge_index=edges, y=label)

    return data, node_positions

# Function to plot the animated graph using Plotly
def plot_graph_animation(df, start_row=0, end_row=10):
    frames = []

    for i in range(start_row, end_row + 1):
        single_row = df.iloc[i]
        _, node_positions = create_data_object_vis(single_row)

        x_coords = [pos[0] for pos in node_positions]
        y_coords = [pos[1] for pos in node_positions]
        z_coords = [pos[2] for pos in node_positions]

        # Create edges for Plotly
        edge_x = []
        edge_y = []
        edge_z = []
        for edge in edges.t().tolist():
            x0, y0, z0 = node_positions[edge[0]]
            x1, y1, z1 = node_positions[edge[1]]
            edge_x.extend([x0, x1, None])
            edge_y.extend([y0, y1, None])
            edge_z.extend([z0, z1, None])

        # Create the scatter and line traces
        edge_trace = go.Scatter3d(
            x=edge_x, y=edge_y, z=edge_z,
            mode='lines',
            line=dict(color='black', width=2),
            hoverinfo='none'
        )

        node_trace = go.Scatter3d(
            x=x_coords, y=y_coords, z=z_coords,
            mode='markers+text',
            marker=dict(size=6, color='blue'),
            text=joints,  # Use the joint names as text labels
            hoverinfo='text'
        )

        # Add the frame to the frames list
        frames.append(go.Frame(data=[edge_trace, node_trace], name=str(i)))

    # Create the figure with frames
    fig = go.Figure(
        data=[frames[0].data[0], frames[0].data[1]],
        layout=go.Layout(
            title='3D Joint Visualization Animation',
            showlegend=False,
            scene=dict(
                xaxis=dict(
                    showbackground=True,
                    backgroundcolor="rgb(230, 230, 230)",
                    gridcolor="rgb(200, 200, 200)",
                    showgrid=True,
                    zerolinecolor="rgb(200, 200, 200)",
                ),
                yaxis=dict(
                    showbackground=True,
                    backgroundcolor="rgb(230, 230, 230)",
                    gridcolor="rgb(200, 200, 200)",
                    showgrid=True,
                    zerolinecolor="rgb(200, 200, 200)",
                ),
                zaxis=dict(
                    showbackground=True,
                    backgroundcolor="rgb(230, 230, 230)",
                    gridcolor="rgb(200, 200, 200)",
                    showgrid=True,
                    zerolinecolor="rgb(200, 200, 200)",
                ),
            ),
            margin=dict(l=0, r=0, b=0, t=40),
            updatemenus=[dict(type="buttons", showactive=False,
                              buttons=[dict(label="Play",
                                            method="animate",
                                            args=[None, dict(frame=dict(duration=500, redraw=True),
                                                             fromcurrent=True)])])]),
        frames=frames
    )

    fig.show()

# Call the function to plot the animation
plot_graph_animation(df, start_row=0, end_row=100)


In [11]:
import plotly.graph_objs as go
import networkx as nx

# Function to create a graph from a single row
def create_graph_from_row(row):
    G = nx.Graph()

    # Define the joints and their coordinates (X, Y, Z)
    joints = {
        'FOOT_RIGHT': (row['FOOT_RIGHT_X'], row['FOOT_RIGHT_Y'], row['FOOT_RIGHT_Z']),
        'FOOT_LEFT': (row['FOOT_LEFT_X'], row['FOOT_LEFT_Y'], row['FOOT_LEFT_Z']),
        'ANKLE_RIGHT': (row['ANKLE_RIGHT_X'], row['ANKLE_RIGHT_Y'], row['ANKLE_RIGHT_Z']),
        'ANKLE_LEFT': (row['ANKLE_LEFT_X'], row['ANKLE_LEFT_Y'], row['ANKLE_LEFT_Z']),
        'KNEE_RIGHT': (row['KNEE_RIGHT_X'], row['KNEE_RIGHT_Y'], row['KNEE_RIGHT_Z']),
        'KNEE_LEFT': (row['KNEE_LEFT_X'], row['KNEE_LEFT_Y'], row['KNEE_LEFT_Z']),
        'HIP_RIGHT': (row['HIP_RIGHT_X'], row['HIP_RIGHT_Y'], row['HIP_RIGHT_Z']),
        'HIP_LEFT': (row['HIP_LEFT_X'], row['HIP_LEFT_Y'], row['HIP_LEFT_Z']),
        'PELVIS': (row['PELVIS_X'], row['PELVIS_Y'], row['PELVIS_Z']),
        'SPINE_NAVAL': (row['SPINE_NAVAL_X'], row['SPINE_NAVAL_Y'], row['SPINE_NAVAL_Z']),
        'SPINE_CHEST': (row['SPINE_CHEST_X'], row['SPINE_CHEST_Y'], row['SPINE_CHEST_Z']),
        'CLAVICLE_RIGHT': (row['CLAVICLE_RIGHT_X'], row['CLAVICLE_RIGHT_Y'], row['CLAVICLE_RIGHT_Z']),
        'CLAVICLE_LEFT': (row['CLAVICLE_LEFT_X'], row['CLAVICLE_LEFT_Y'], row['CLAVICLE_LEFT_Z']),
        'SHOULDER_RIGHT': (row['SHOULDER_RIGHT_X'], row['SHOULDER_RIGHT_Y'], row['SHOULDER_RIGHT_Z']),
        'SHOULDER_LEFT': (row['SHOULDER_LEFT_X'], row['SHOULDER_LEFT_Y'], row['SHOULDER_LEFT_Z']),
        'ELBOW_RIGHT': (row['ELBOW_RIGHT_X'], row['ELBOW_RIGHT_Y'], row['ELBOW_RIGHT_Z']),
        'ELBOW_LEFT': (row['ELBOW_LEFT_X'], row['ELBOW_LEFT_Y'], row['ELBOW_LEFT_Z']),
        'WRIST_RIGHT': (row['WRIST_RIGHT_X'], row['WRIST_RIGHT_Y'], row['WRIST_RIGHT_Z']),
        'WRIST_LEFT': (row['WRIST_LEFT_X'], row['WRIST_LEFT_Y'], row['WRIST_LEFT_Z']),
        'HAND_RIGHT': (row['HAND_RIGHT_X'], row['HAND_RIGHT_Y'], row['HAND_RIGHT_Z']),
        'HAND_LEFT': (row['HAND_LEFT_X'], row['HAND_LEFT_Y'], row['HAND_LEFT_Z']),
        'HANDTIP_RIGHT': (row['HANDTIP_RIGHT_X'], row['HANDTIP_RIGHT_Y'], row['HANDTIP_RIGHT_Z']),
        'HANDTIP_LEFT': (row['HANDTIP_LEFT_X'], row['HANDTIP_LEFT_Y'], row['HANDTIP_LEFT_Z']),
        'THUMB_RIGHT': (row['THUMB_RIGHT_X'], row['THUMB_RIGHT_Y'], row['THUMB_RIGHT_Z']),
        'THUMB_LEFT': (row['THUMB_LEFT_X'], row['THUMB_LEFT_Y'], row['THUMB_LEFT_Z']),
        'NECK': (row['NECK_X'], row['NECK_Y'], row['NECK_Z']),
        'HEAD': (row['HEAD_X'], row['HEAD_Y'], row['HEAD_Z']),
    }

    # Add nodes to the graph
    for joint, coords in joints.items():
        G.add_node(joint, pos=coords)

    # Define edges based on anatomical connections
    G.add_edges_from([
        # Legs
        ('FOOT_RIGHT', 'ANKLE_RIGHT'),
        ('FOOT_LEFT', 'ANKLE_LEFT'),
        ('ANKLE_RIGHT', 'KNEE_RIGHT'),
        ('ANKLE_LEFT', 'KNEE_LEFT'),
        ('KNEE_RIGHT', 'HIP_RIGHT'),
        ('KNEE_LEFT', 'HIP_LEFT'),
        ('HIP_RIGHT', 'PELVIS'),
        ('HIP_LEFT', 'PELVIS'),

        # Spine and arms
        ('PELVIS', 'SPINE_NAVAL'),
        ('SPINE_NAVAL', 'SPINE_CHEST'),
        ('SPINE_CHEST', 'CLAVICLE_RIGHT'),
        ('SPINE_CHEST', 'CLAVICLE_LEFT'),
        ('CLAVICLE_RIGHT', 'SHOULDER_RIGHT'),
        ('CLAVICLE_LEFT', 'SHOULDER_LEFT'),
        ('SHOULDER_RIGHT', 'ELBOW_RIGHT'),
        ('SHOULDER_LEFT', 'ELBOW_LEFT'),
        ('ELBOW_RIGHT', 'WRIST_RIGHT'),
        ('ELBOW_LEFT', 'WRIST_LEFT'),
        ('WRIST_RIGHT', 'HAND_RIGHT'),
        ('WRIST_LEFT', 'HAND_LEFT'),
        ('HAND_RIGHT', 'HANDTIP_RIGHT'),
        ('HAND_LEFT', 'HANDTIP_LEFT'),
        ('HANDTIP_RIGHT', 'THUMB_RIGHT'),
        ('HANDTIP_LEFT', 'THUMB_LEFT'),

        # Head and neck
        ('SPINE_CHEST', 'NECK'),
        ('NECK', 'HEAD'),
    ])

    return G

# Function to visualize the graph in 3D with Plotly
def visualize_graph_3d_plotly(G):
    pos = nx.get_node_attributes(G, 'pos')

    # Create edges
    edge_x = []
    edge_y = []
    edge_z = []
    for edge in G.edges():
        x0, y0, z0 = pos[edge[0]]
        x1, y1, z1 = pos[edge[1]]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])
        edge_z.extend([z0, z1, None])

    edge_trace = go.Scatter3d(
        x=edge_x, y=edge_y, z=edge_z,
        line=dict(width=2, color='black'),
        hoverinfo='none',
        mode='lines')

    # Create nodes
    node_x = []
    node_y = []
    node_z = []
    node_text = []
    for node in G.nodes():
        x, y, z = pos[node]
        node_x.append(x)
        node_y.append(y)
        node_z.append(z)
        node_text.append(node)

    node_trace = go.Scatter3d(
        x=node_x, y=node_y, z=node_z,
        mode='markers+text',
        marker=dict(
            size=8,
            color='blue',
            line=dict(width=2)),
        text=node_text,
        hoverinfo='text')

    # Create the figure with gridlines
    fig = go.Figure(data=[edge_trace, node_trace],
                    layout=go.Layout(
                        title='3D Human Skeleton Visualization with Grid',
                        showlegend=False,
                        scene=dict(
                            xaxis=dict(
                                showbackground=True,
                                backgroundcolor="rgb(230, 230, 230)",
                                gridcolor="rgb(200, 200, 200)",
                                showgrid=True,
                                zerolinecolor="rgb(200, 200, 200)",
                            ),
                            yaxis=dict(
                                showbackground=True,
                                backgroundcolor="rgb(230, 230, 230)",
                                gridcolor="rgb(200, 200, 200)",
                                showgrid=True,
                                zerolinecolor="rgb(200, 200, 200)",
                            ),
                            zaxis=dict(
                                showbackground=True,
                                backgroundcolor="rgb(230, 230, 230)",
                                gridcolor="rgb(200, 200, 200)",
                                showgrid=True,
                                zerolinecolor="rgb(200, 200, 200)",
                            ),
                        ),
                        margin=dict(l=0, r=0, b=0, t=40),
                    ))

    fig.show()

# Example usage
# Replace df.iloc[10] with an actual DataFrame row containing the skeleton data
G = create_graph_from_row(df.iloc[10])
visualize_graph_3d_plotly(G)


In [15]:
import torch
import pandas as pd
import plotly.graph_objs as go
from torch_geometric.data import Data

# Load the dataset

df = pd.read_csv(file_path)

# Define anatomical connections as edges
edges = torch.tensor([
    [0, 2],  # FOOT_RIGHT to ANKLE_RIGHT
    [1, 3],  # FOOT_LEFT to ANKLE_LEFT
    [2, 4],  # ANKLE_RIGHT to KNEE_RIGHT
    [3, 5],  # ANKLE_LEFT to KNEE_LEFT
    [4, 6],  # KNEE_RIGHT to HIP_RIGHT
    [5, 7],  # KNEE_LEFT to HIP_LEFT
    [6, 8],  # HIP_RIGHT to PELVIS
    [7, 8],  # HIP_LEFT to PELVIS
    [8, 9],  # PELVIS to SPINE_NAVAL
    [9, 10], # SPINE_NAVAL to SPINE_CHEST
    [10, 11],# SPINE_CHEST to CLAVICLE_RIGHT
    [10, 12],# SPINE_CHEST to CLAVICLE_LEFT
    [11, 13],# CLAVICLE_RIGHT to SHOULDER_RIGHT
    [12, 14],# CLAVICLE_LEFT to SHOULDER_LEFT
    [13, 15],# SHOULDER_RIGHT to ELBOW_RIGHT
    [14, 16],# SHOULDER_LEFT to ELBOW_LEFT
    [15, 17],# ELBOW_RIGHT to WRIST_RIGHT
    [16, 18],# ELBOW_LEFT to WRIST_LEFT
    [17, 19],# WRIST_RIGHT to HAND_RIGHT
    [18, 20],# WRIST_LEFT to HAND_LEFT
    [19, 21],# HAND_RIGHT to HANDTIP_RIGHT
    [20, 22],# HAND_LEFT to HANDTIP_LEFT
    [21, 23],# HANDTIP_RIGHT to THUMB_RIGHT
    [22, 24],# HANDTIP_LEFT to THUMB_LEFT
    [10, 25],# SPINE_CHEST to NECK
    [25, 26],# NECK to HEAD
]).t().contiguous()

# List of joints in the order they appear in the DataFrame
joints = [
    'FOOT_RIGHT', 'FOOT_LEFT', 'ANKLE_RIGHT', 'ANKLE_LEFT', 'KNEE_RIGHT', 'KNEE_LEFT',
    'HIP_RIGHT', 'HIP_LEFT', 'PELVIS', 'SPINE_NAVAL', 'SPINE_CHEST',
    'CLAVICLE_RIGHT', 'CLAVICLE_LEFT', 'SHOULDER_RIGHT', 'SHOULDER_LEFT',
    'ELBOW_RIGHT', 'ELBOW_LEFT', 'WRIST_RIGHT', 'WRIST_LEFT', 'HAND_RIGHT',
    'HAND_LEFT', 'HANDTIP_RIGHT', 'HANDTIP_LEFT', 'THUMB_RIGHT', 'THUMB_LEFT',
    'NECK', 'HEAD', 'NOSE', 'EYE_LEFT', 'EAR_LEFT', 'EYE_RIGHT', 'EAR_RIGHT'
]

# Function to extract node features and create a Data object
def create_data_object_vis(row):
    node_features = []
    node_positions = []  # Store (x, y, z) positions separately for Plotly visualization

    for joint in joints:
        x = row[f'{joint}_X']
        y = row[f'{joint}_Y']
        z = row[f'{joint}_Z']
        node_features.append([x, y, z])
        node_positions.append((x, y, z))  # Save position for each joint

    node_features = torch.tensor(node_features, dtype=torch.float)

    # Extract label (assuming 'frailty' is the target column)
    label = torch.tensor([row['QoR_class']], dtype=torch.float)

    # Create the Data object
    data = Data(x=node_features, edge_index=edges, y=label)

    return data, node_positions

# Function to plot the graph using Plotly
def plot_graph(node_positions):
    # Extract x, y, z coordinates from node_positions
    x_coords = [pos[0] for pos in node_positions]
    y_coords = [pos[1] for pos in node_positions]
    z_coords = [pos[2] for pos in node_positions]

    # Create edges for Plotly
    edge_x = []
    edge_y = []
    edge_z = []
    for edge in edges.t().tolist():
        x0, y0, z0 = node_positions[edge[0]]
        x1, y1, z1 = node_positions[edge[1]]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])
        edge_z.extend([z0, z1, None])

    # Plot the edges
    edge_trace = go.Scatter3d(
        x=edge_x, y=edge_y, z=edge_z,
        mode='lines',
        line=dict(color='black', width=2),
        hoverinfo='none'
    )

    # Plot the nodes with joint names
    node_trace = go.Scatter3d(
        x=x_coords, y=y_coords, z=z_coords,
        mode='markers+text',
        marker=dict(size=6, color='blue'),
        text=joints,  # Use the joint names as text labels
        hoverinfo='text'
    )

    # Create the figure with gridlines and joint names
    fig = go.Figure(data=[edge_trace, node_trace],
                    layout=go.Layout(
                        title='3D  Visualization',
                        showlegend=False,
                        scene=dict(
                            xaxis=dict(
                                showbackground=True,
                                backgroundcolor="rgb(230, 230, 230)",
                                gridcolor="rgb(200, 200, 200)",
                                showgrid=True,
                                zerolinecolor="rgb(200, 200, 200)",
                            ),
                            yaxis=dict(
                                showbackground=True,
                                backgroundcolor="rgb(230, 230, 230)",
                                gridcolor="rgb(200, 200, 200)",
                                showgrid=True,
                                zerolinecolor="rgb(200, 200, 200)",
                            ),
                            zaxis=dict(
                                showbackground=True,
                                backgroundcolor="rgb(230, 230, 230)",
                                gridcolor="rgb(200, 200, 200)",
                                showgrid=True,
                                zerolinecolor="rgb(200, 200, 200)",
                            ),
                        ),
                        margin=dict(l=0, r=0, b=0, t=40),
                    ))

    fig.show()

# Example usage with a single row (e.g., the first row in your DataFrame)
single_row = df.iloc[0]
data, node_positions = create_data_object_vis(single_row)

# Plot the graph using Plotly
plot_graph(node_positions)


In [16]:
import torch
import pandas as pd
import plotly.graph_objs as go
from torch_geometric.data import Data

# Load the dataset

df = pd.read_csv(file_path)
"""
# Define anatomical connections as edges
edges = torch.tensor([
    [0, 1],   # PELVIS to SPINE_NAVAL
    [1, 2],   # SPINE_NAVAL to SPINE_CHEST
    [2, 3],   # SPINE_CHEST to NECK
    [2, 4],   # SPINE_CHEST to CLAVICLE_LEFT
    [4, 5],   # CLAVICLE_LEFT to SHOULDER_LEFT
    [5, 6],   # SHOULDER_LEFT to ELBOW_LEFT
    [6, 7],   # ELBOW_LEFT to WRIST_LEFT
    [7, 8],   # WRIST_LEFT to HAND_LEFT
    [8, 9],   # HAND_LEFT to HANDTIP_LEFT
    [7, 10],  # WRIST_LEFT to THUMB_LEFT
    [2, 11],  # SPINE_CHEST to CLAVICLE_RIGHT
    [11, 12], # CLAVICLE_RIGHT to SHOULDER_RIGHT
    [12, 13], # SHOULDER_RIGHT to ELBOW_RIGHT
    [13, 14], # ELBOW_RIGHT to WRIST_RIGHT
    [14, 15], # WRIST_RIGHT to HAND_RIGHT
    [15, 16], # HAND_RIGHT to HANDTIP_RIGHT
    [14, 17], # WRIST_RIGHT to THUMB_RIGHT
    [0, 18],  # PELVIS to HIP_LEFT
    [18, 19], # HIP_LEFT to KNEE_LEFT
    [19, 20], # KNEE_LEFT to ANKLE_LEFT
    [20, 21], # ANKLE_LEFT to FOOT_LEFT
    [0, 22],  # PELVIS to HIP_RIGHT
    [22, 23], # HIP_RIGHT to KNEE_RIGHT
    [23, 24], # KNEE_RIGHT to ANKLE_RIGHT
    [24, 25], # ANKLE_RIGHT to FOOT_RIGHT
]).t().contiguous()

# List of joints in the order they appear in the DataFrame
joints = [
    'FOOT_RIGHT', 'FOOT_LEFT', 'ANKLE_RIGHT', 'ANKLE_LEFT', 'KNEE_RIGHT', 'KNEE_LEFT',
    'HIP_RIGHT', 'HIP_LEFT', 'PELVIS', 'SPINE_NAVAL', 'SPINE_CHEST',
    'CLAVICLE_RIGHT', 'CLAVICLE_LEFT', 'SHOULDER_RIGHT', 'SHOULDER_LEFT',
    'ELBOW_RIGHT', 'ELBOW_LEFT', 'WRIST_RIGHT', 'WRIST_LEFT', 'HAND_RIGHT',
    'HAND_LEFT', 'HANDTIP_RIGHT', 'HANDTIP_LEFT', 'THUMB_RIGHT', 'THUMB_LEFT',
    'NECK', 'HEAD', 'NOSE', 'EYE_LEFT', 'EAR_LEFT', 'EYE_RIGHT', 'EAR_RIGHT'
]
"""

edges = torch.tensor([
    [0, 1],   # PELVIS to SPINE_NAVAL
    [1, 2],   # SPINE_NAVAL to SPINE_CHEST
    [2, 3],   # SPINE_CHEST to NECK
    [2, 4],   # SPINE_CHEST to CLAVICLE_LEFT
    [4, 5],   # CLAVICLE_LEFT to SHOULDER_LEFT
    [5, 6],   # SHOULDER_LEFT to ELBOW_LEFT
    [6, 7],   # ELBOW_LEFT to WRIST_LEFT
    [7, 8],   # WRIST_LEFT to HAND_LEFT
    [8, 9],   # HAND_LEFT to HANDTIP_LEFT
    [7, 10],  # WRIST_LEFT to THUMB_LEFT
    [2, 11],  # SPINE_CHEST to CLAVICLE_RIGHT
    [11, 12], # CLAVICLE_RIGHT to SHOULDER_RIGHT
    [12, 13], # SHOULDER_RIGHT to ELBOW_RIGHT
    [13, 14], # ELBOW_RIGHT to WRIST_RIGHT
    [14, 15], # WRIST_RIGHT to HAND_RIGHT
    [15, 16], # HAND_RIGHT to HANDTIP_RIGHT
    [14, 17], # WRIST_RIGHT to THUMB_RIGHT
    [0, 18],  # PELVIS to HIP_LEFT
    [18, 19], # HIP_LEFT to KNEE_LEFT
    [19, 20], # KNEE_LEFT to ANKLE_LEFT
    [20, 21], # ANKLE_LEFT to FOOT_LEFT
    [0, 22],  # PELVIS to HIP_RIGHT
    [22, 23], # HIP_RIGHT to KNEE_RIGHT
    [23, 24], # KNEE_RIGHT to ANKLE_RIGHT
    [24, 25], # ANKLE_RIGHT to FOOT_RIGHT
]).t().contiguous()

joints = [
    'PELVIS', 'SPINE_NAVAL', 'SPINE_CHEST', 'NECK', 'CLAVICLE_LEFT', 'SHOULDER_LEFT',
    'ELBOW_LEFT', 'WRIST_LEFT', 'HAND_LEFT', 'HANDTIP_LEFT', 'THUMB_LEFT',
    'CLAVICLE_RIGHT', 'SHOULDER_RIGHT', 'ELBOW_RIGHT', 'WRIST_RIGHT', 'HAND_RIGHT',
    'HANDTIP_RIGHT', 'THUMB_RIGHT', 'HIP_LEFT', 'KNEE_LEFT', 'ANKLE_LEFT',
    'FOOT_LEFT', 'HIP_RIGHT', 'KNEE_RIGHT', 'ANKLE_RIGHT', 'FOOT_RIGHT'
]


# Function to extract node features and create a Data object
def create_data_object_vis(row):
    node_features = []
    node_positions = []  # Store (x, y, z) positions separately for Plotly visualization

    for joint in joints:
        x = row[f'{joint}_X']
        y = row[f'{joint}_Y']
        z = row[f'{joint}_Z']
        node_features.append([x, y, z])
        node_positions.append((x, y, z))  # Save position for each joint

    node_features = torch.tensor(node_features, dtype=torch.float)

    # Extract label (assuming 'frailty' is the target column)
    label = torch.tensor([row['QoR_class']], dtype=torch.float)

    # Create the Data object
    data = Data(x=node_features, edge_index=edges, y=label)

    return data, node_positions

# Function to plot the graph using Plotly
def plot_graph(node_positions):
    # Extract x, y, z coordinates from node_positions
    x_coords = [pos[0] for pos in node_positions]
    y_coords = [pos[1] for pos in node_positions]
    z_coords = [pos[2] for pos in node_positions]

    # Create edges for Plotly
    edge_x = []
    edge_y = []
    edge_z = []
    for edge in edges.t().tolist():
        x0, y0, z0 = node_positions[edge[0]]
        x1, y1, z1 = node_positions[edge[1]]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])
        edge_z.extend([z0, z1, None])

    # Plot the edges
    edge_trace = go.Scatter3d(
        x=edge_x, y=edge_y, z=edge_z,
        mode='lines',
        line=dict(color='black', width=2),
        hoverinfo='none'
    )

    # Plot the nodes with joint names
    node_trace = go.Scatter3d(
        x=x_coords, y=y_coords, z=z_coords,
        mode='markers+text',
        marker=dict(size=6, color='blue'),
        text=joints,  # Use the joint names as text labels
        hoverinfo='text'
    )

    # Create the figure with gridlines and joint names
    fig = go.Figure(data=[edge_trace, node_trace],
                    layout=go.Layout(
                        title='3D  Visualization',
                        showlegend=False,
                        scene=dict(
                            xaxis=dict(
                                showbackground=True,
                                backgroundcolor="rgb(230, 230, 230)",
                                gridcolor="rgb(200, 200, 200)",
                                showgrid=True,
                                zerolinecolor="rgb(200, 200, 200)",
                            ),
                            yaxis=dict(
                                showbackground=True,
                                backgroundcolor="rgb(230, 230, 230)",
                                gridcolor="rgb(200, 200, 200)",
                                showgrid=True,
                                zerolinecolor="rgb(200, 200, 200)",
                            ),
                            zaxis=dict(
                                showbackground=True,
                                backgroundcolor="rgb(230, 230, 230)",
                                gridcolor="rgb(200, 200, 200)",
                                showgrid=True,
                                zerolinecolor="rgb(200, 200, 200)",
                            ),
                        ),
                        margin=dict(l=0, r=0, b=0, t=40),
                    ))

    fig.show()

# Example usage with a single row (e.g., the first row in your DataFrame)
single_row = df.iloc[0]
data, node_positions = create_data_object_vis(single_row)

# Plot the graph using Plotly
plot_graph(node_positions)


In [17]:
import torch
import pandas as pd
import plotly.graph_objs as go
from torch_geometric.data import Data

# Load the dataset

df = pd.read_csv(file_path)

# Define anatomical connections as edges
edges = torch.tensor([
    [0, 2],  # FOOT_RIGHT to ANKLE_RIGHT
    [1, 3],  # FOOT_LEFT to ANKLE_LEFT
    [2, 4],  # ANKLE_RIGHT to KNEE_RIGHT
    [3, 5],  # ANKLE_LEFT to KNEE_LEFT
    [4, 6],  # KNEE_RIGHT to HIP_RIGHT
    [5, 7],  # KNEE_LEFT to HIP_LEFT
    [6, 8],  # HIP_RIGHT to PELVIS
    [7, 8],  # HIP_LEFT to PELVIS
    [8, 9],  # PELVIS to SPINE_NAVAL
    [9, 10], # SPINE_NAVAL to SPINE_CHEST
    [10, 11],# SPINE_CHEST to CLAVICLE_RIGHT
    [10, 12],# SPINE_CHEST to CLAVICLE_LEFT
    [11, 13],# CLAVICLE_RIGHT to SHOULDER_RIGHT
    [12, 14],# CLAVICLE_LEFT to SHOULDER_LEFT
    [13, 15],# SHOULDER_RIGHT to ELBOW_RIGHT
    [14, 16],# SHOULDER_LEFT to ELBOW_LEFT
    [15, 17],# ELBOW_RIGHT to WRIST_RIGHT
    [16, 18],# ELBOW_LEFT to WRIST_LEFT
    [17, 19],# WRIST_RIGHT to HAND_RIGHT
    [18, 20],# WRIST_LEFT to HAND_LEFT
    [19, 21],# HAND_RIGHT to HANDTIP_RIGHT
    [20, 22],# HAND_LEFT to HANDTIP_LEFT
    [21, 23],# HANDTIP_RIGHT to THUMB_RIGHT
    [22, 24],# HANDTIP_LEFT to THUMB_LEFT
    [10, 25],# SPINE_CHEST to NECK
    [25, 26],# NECK to HEAD
]).t().contiguous()


edges = torch.tensor([
    [0, 2],  # FOOT_RIGHT to ANKLE_RIGHT
    [1, 3],  # FOOT_LEFT to ANKLE_LEFT
    [2, 4],  # ANKLE_RIGHT to KNEE_RIGHT
    [3, 5],  # ANKLE_LEFT to KNEE_LEFT
    [4, 6],  # KNEE_RIGHT to HIP_RIGHT
    [5, 7],  # KNEE_LEFT to HIP_LEFT
    [6, 8],  # HIP_RIGHT to PELVIS
    [7, 8],  # HIP_LEFT to PELVIS
]).t().contiguous()



# List of joints in the order they appear in the DataFrame
joints = [
    'FOOT_RIGHT', 'FOOT_LEFT', 'ANKLE_RIGHT', 'ANKLE_LEFT', 'KNEE_RIGHT', 'KNEE_LEFT',
    'HIP_RIGHT', 'HIP_LEFT', 'PELVIS', 'SPINE_NAVAL', 'SPINE_CHEST',
    'CLAVICLE_RIGHT', 'CLAVICLE_LEFT', 'SHOULDER_RIGHT', 'SHOULDER_LEFT',
    'ELBOW_RIGHT', 'ELBOW_LEFT', 'WRIST_RIGHT', 'WRIST_LEFT', 'HAND_RIGHT',
    'HAND_LEFT', 'HANDTIP_RIGHT', 'HANDTIP_LEFT', 'THUMB_RIGHT', 'THUMB_LEFT',
    'NECK', 'HEAD', 'NOSE', 'EYE_LEFT', 'EAR_LEFT', 'EYE_RIGHT', 'EAR_RIGHT'
]


# List of joints in the order they appear in the DataFrame
joints = [
    'FOOT_RIGHT', 'FOOT_LEFT', 'ANKLE_RIGHT', 'ANKLE_LEFT', 'KNEE_RIGHT', 'KNEE_LEFT',
    'HIP_RIGHT', 'HIP_LEFT', 'PELVIS'
]

# Function to extract node features and create a Data object
def create_data_object_vis(row):
    node_features = []
    node_positions = []  # Store (x, y, z) positions separately for Plotly visualization

    for joint in joints:
        x = row[f'{joint}_X']
        y = row[f'{joint}_Y']
        z = row[f'{joint}_Z']
        node_features.append([x, y, z])
        node_positions.append((x, y, z))  # Save position for each joint

    node_features = torch.tensor(node_features, dtype=torch.float)

    # Extract label (assuming 'frailty' is the target column)
    label = torch.tensor([row['QoR_class']], dtype=torch.float)

    # Create the Data object
    data = Data(x=node_features, edge_index=edges, y=label)

    return data, node_positions

# Function to plot the graph using Plotly
def plot_graph(node_positions):
    # Extract x, y, z coordinates from node_positions
    x_coords = [pos[0] for pos in node_positions]
    y_coords = [pos[1] for pos in node_positions]
    z_coords = [pos[2] for pos in node_positions]

    # Create edges for Plotly
    edge_x = []
    edge_y = []
    edge_z = []
    for edge in edges.t().tolist():
        x0, y0, z0 = node_positions[edge[0]]
        x1, y1, z1 = node_positions[edge[1]]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])
        edge_z.extend([z0, z1, None])

    # Plot the edges
    edge_trace = go.Scatter3d(
        x=edge_x, y=edge_y, z=edge_z,
        mode='lines',
        line=dict(color='black', width=2),
        hoverinfo='none'
    )

    # Plot the nodes with joint names and coordinates
    node_trace = go.Scatter3d(
        x=x_coords, y=y_coords, z=z_coords,
        mode='markers+text',
        marker=dict(size=6, color='blue'),
        text=[f'{joint}<br>({x:.2f}, {y:.2f}, {z:.2f})' for joint, x, y, z in zip(joints, x_coords, y_coords, z_coords)],  # Joint names and coordinates
        hoverinfo='text'
    )

    # Create the figure with gridlines and joint names
    fig = go.Figure(data=[edge_trace, node_trace],
                    layout=go.Layout(
                        title='3D Joint Visualization',
                        showlegend=False,
                        scene=dict(
                            xaxis=dict(
                                showbackground=True,
                                backgroundcolor="rgb(230, 230, 230)",
                                gridcolor="rgb(200, 200, 200)",
                                showgrid=True,
                                zerolinecolor="rgb(200, 200, 200)",
                            ),
                            yaxis=dict(
                                showbackground=True,
                                backgroundcolor="rgb(230, 230, 230)",
                                gridcolor="rgb(200, 200, 200)",
                                showgrid=True,
                                zerolinecolor="rgb(200, 200, 200)",
                            ),
                            zaxis=dict(
                                showbackground=True,
                                backgroundcolor="rgb(230, 230, 230)",
                                gridcolor="rgb(200, 200, 200)",
                                showgrid=True,
                                zerolinecolor="rgb(200, 200, 200)",
                            ),
                        ),
                        margin=dict(l=0, r=0, b=0, t=40),
                    ))

    fig.show()

# Example usage with a single row (e.g., the first row in your DataFrame)
single_row = df.iloc[0]
data, node_positions = create_data_object_vis(single_row)

# Plot the graph using Plotly
plot_graph(node_positions)


In [18]:
import torch
import pandas as pd
import plotly.graph_objs as go
from torch_geometric.data import Data

# Load the dataset

df = pd.read_csv(file_path)

# Define anatomical connections as edges
edges = torch.tensor([
    [0, 2],  # FOOT_RIGHT to ANKLE_RIGHT
    [1, 3],  # FOOT_LEFT to ANKLE_LEFT
    [2, 4],  # ANKLE_RIGHT to KNEE_RIGHT
    [3, 5],  # ANKLE_LEFT to KNEE_LEFT
    [4, 6],  # KNEE_RIGHT to HIP_RIGHT
    [5, 7],  # KNEE_LEFT to HIP_LEFT
    [6, 8],  # HIP_RIGHT to PELVIS
    [7, 8],  # HIP_LEFT to PELVIS
    [8, 9],  # PELVIS to SPINE_NAVAL
    [9, 10], # SPINE_NAVAL to SPINE_CHEST
    [10, 11],# SPINE_CHEST to CLAVICLE_RIGHT
    [10, 12],# SPINE_CHEST to CLAVICLE_LEFT
    [11, 13],# CLAVICLE_RIGHT to SHOULDER_RIGHT
    [12, 14],# CLAVICLE_LEFT to SHOULDER_LEFT
    [13, 15],# SHOULDER_RIGHT to ELBOW_RIGHT
    [14, 16],# SHOULDER_LEFT to ELBOW_LEFT
    [15, 17],# ELBOW_RIGHT to WRIST_RIGHT
    [16, 18],# ELBOW_LEFT to WRIST_LEFT
    [17, 19],# WRIST_RIGHT to HAND_RIGHT
    [18, 20],# WRIST_LEFT to HAND_LEFT
    [19, 21],# HAND_RIGHT to HANDTIP_RIGHT
    [20, 22],# HAND_LEFT to HANDTIP_LEFT
    [21, 23],# HANDTIP_RIGHT to THUMB_RIGHT
    [22, 24],# HANDTIP_LEFT to THUMB_LEFT
    [10, 25],# SPINE_CHEST to NECK
    [25, 26],# NECK to HEAD
]).t().contiguous()


edges = torch.tensor([
    [0, 2],  # FOOT_RIGHT to ANKLE_RIGHT
    [1, 3],  # FOOT_LEFT to ANKLE_LEFT
    [2, 4],  # ANKLE_RIGHT to KNEE_RIGHT
    [3, 5],  # ANKLE_LEFT to KNEE_LEFT
    [4, 6],  # KNEE_RIGHT to HIP_RIGHT
    [5, 7],  # KNEE_LEFT to HIP_LEFT
    [6, 8],  # HIP_RIGHT to PELVIS
    [7, 8],  # HIP_LEFT to PELVIS
]).t().contiguous()

# List of joints in the order they appear in the DataFrame
joints = [
    'FOOT_RIGHT', 'FOOT_LEFT', 'ANKLE_RIGHT', 'ANKLE_LEFT', 'KNEE_RIGHT', 'KNEE_LEFT',
    'HIP_RIGHT', 'HIP_LEFT', 'PELVIS', 'SPINE_NAVAL', 'SPINE_CHEST',
    'CLAVICLE_RIGHT', 'CLAVICLE_LEFT', 'SHOULDER_RIGHT', 'SHOULDER_LEFT',
    'ELBOW_RIGHT', 'ELBOW_LEFT', 'WRIST_RIGHT', 'WRIST_LEFT', 'HAND_RIGHT',
    'HAND_LEFT', 'HANDTIP_RIGHT', 'HANDTIP_LEFT', 'THUMB_RIGHT', 'THUMB_LEFT',
    'NECK', 'HEAD', 'NOSE', 'EYE_LEFT', 'EAR_LEFT', 'EYE_RIGHT', 'EAR_RIGHT'
]


joints = [
    'FOOT_RIGHT', 'FOOT_LEFT', 'ANKLE_RIGHT', 'ANKLE_LEFT', 'KNEE_RIGHT', 'KNEE_LEFT',
    'HIP_RIGHT', 'HIP_LEFT', 'PELVIS'
]


# Function to extract node features and create a Data object
def create_data_object_vis(row):
    node_features = []
    node_positions = []  # Store (x, y, z) positions separately for Plotly visualization

    for joint in joints:
        x = row[f'{joint}_X']
        y = row[f'{joint}_Y']
        z = row[f'{joint}_Z']
        node_features.append([x, y, z])
        node_positions.append((x, y, z))  # Save position for each joint

    node_features = torch.tensor(node_features, dtype=torch.float)

    # Extract label (assuming 'frailty' is the target column)
    label = torch.tensor([row['QoR_class']], dtype=torch.float)

    # Create the Data object
    data = Data(x=node_features, edge_index=edges, y=label)

    return data, node_positions

# Function to plot the graph using Plotly
def plot_graph(node_positions):
    # Extract x, y, z coordinates from node_positions
    x_coords = [pos[0] for pos in node_positions]
    y_coords = [pos[1] for pos in node_positions]
    z_coords = [pos[2] for pos in node_positions]

    # Create edges for Plotly
    edge_x = []
    edge_y = []
    edge_z = []
    for edge in edges.t().tolist():
        x0, y0, z0 = node_positions[edge[0]]
        x1, y1, z1 = node_positions[edge[1]]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])
        edge_z.extend([z0, z1, None])

    # Plot the edges
    edge_trace = go.Scatter3d(
        x=edge_x, y=edge_y, z=edge_z,
        mode='lines',
        line=dict(color='black', width=2),
        hoverinfo='none'
    )

    # Plot the nodes with and without coordinates
    node_trace_with_coords = go.Scatter3d(
        x=x_coords, y=y_coords, z=z_coords,
        mode='markers+text',
        marker=dict(size=6, color='blue'),
        text=[f'{joint}<br>({x:.2f}, {y:.2f}, {z:.2f})' for joint, x, y, z in zip(joints, x_coords, y_coords, z_coords)],
        hoverinfo='text',
        name="With Coordinates"
    )

    node_trace_without_coords = go.Scatter3d(
        x=x_coords, y=y_coords, z=z_coords,
        mode='markers',
        marker=dict(size=6, color='blue'),
        hoverinfo='none',
        name="Without Coordinates"
    )

    # Create the figure with gridlines and joint names
    fig = go.Figure(data=[edge_trace, node_trace_with_coords, node_trace_without_coords],
                    layout=go.Layout(
                        title='3D Joint Visualization',
                        showlegend=False,
                        scene=dict(
                            xaxis=dict(
                                showbackground=True,
                                backgroundcolor="rgb(230, 230, 230)",
                                gridcolor="rgb(200, 200, 200)",
                                showgrid=True,
                                zerolinecolor="rgb(200, 200, 200)",
                            ),
                            yaxis=dict(
                                showbackground=True,
                                backgroundcolor="rgb(230, 230, 230)",
                                gridcolor="rgb(200, 200, 200)",
                                showgrid=True,
                                zerolinecolor="rgb(200, 200, 200)",
                            ),
                            zaxis=dict(
                                showbackground=True,
                                backgroundcolor="rgb(230, 230, 230)",
                                gridcolor="rgb(200, 200, 200)",
                                showgrid=True,
                                zerolinecolor="rgb(200, 200, 200)",
                            ),
                        ),
                        margin=dict(l=0, r=0, b=0, t=40),
                        updatemenus=[dict(
                            type="buttons",
                            showactive=True,
                            buttons=list([
                                dict(label="Show Coordinates",
                                     method="update",
                                     args=[{"visible": [True, True, False]},  # Show edges and node trace with coordinates, hide without coordinates
                                           {"title": "3D Joint Visualization with Coordinates"}]),
                                dict(label="Hide Coordinates",
                                     method="update",
                                     args=[{"visible": [True, False, True]},  # Show edges and node trace without coordinates, hide with coordinates
                                           {"title": "3D Joint Visualization without Coordinates"}])
                            ]),
                            x=0.17,  # Position of the button
                            xanchor="left",
                            y=1.12,
                            yanchor="top"
                        )]
                    ))

    fig.show()

# Example usage with a single row (e.g., the first row in your DataFrame)
single_row = df.iloc[0]
data, node_positions = create_data_object_vis(single_row)

# Plot the graph using Plotly
plot_graph(node_positions)


In [19]:
import torch
import pandas as pd
import plotly.graph_objs as go
from torch_geometric.data import Data

# Define anatomical connections as edges
edges = torch.tensor([
    [0, 2],  # FOOT_RIGHT to ANKLE_RIGHT
    [1, 3],  # FOOT_LEFT to ANKLE_LEFT
    [2, 4],  # ANKLE_RIGHT to KNEE_RIGHT
    [3, 5],  # ANKLE_LEFT to KNEE_LEFT
    [4, 6],  # KNEE_RIGHT to HIP_RIGHT
    [5, 7],  # KNEE_LEFT to HIP_LEFT
    [6, 8],  # HIP_RIGHT to PELVIS
    [7, 8],  # HIP_LEFT to PELVIS
    [8, 9],  # PELVIS to SPINE_NAVAL
    [9, 10], # SPINE_NAVAL to SPINE_CHEST
    [10, 11],# SPINE_CHEST to CLAVICLE_RIGHT
    [10, 12],# SPINE_CHEST to CLAVICLE_LEFT
    [11, 13],# CLAVICLE_RIGHT to SHOULDER_RIGHT
    [12, 14],# CLAVICLE_LEFT to SHOULDER_LEFT
    [13, 15],# SHOULDER_RIGHT to ELBOW_RIGHT
    [14, 16],# SHOULDER_LEFT to ELBOW_LEFT
    [15, 17],# ELBOW_RIGHT to WRIST_RIGHT
    [16, 18],# ELBOW_LEFT to WRIST_LEFT
    [17, 19],# WRIST_RIGHT to HAND_RIGHT
    [18, 20],# WRIST_LEFT to HAND_LEFT
    [19, 21],# HAND_RIGHT to HANDTIP_RIGHT
    [20, 22],# HAND_LEFT to HANDTIP_LEFT
    [21, 23],# HANDTIP_RIGHT to THUMB_RIGHT
    [22, 24],# HANDTIP_LEFT to THUMB_LEFT
    [10, 25],# SPINE_CHEST to NECK
    [25, 26],# NECK to HEAD
]).t().contiguous()

# List of joints in the order they appear
joints = [
    'FOOT_RIGHT', 'FOOT_LEFT', 'ANKLE_RIGHT', 'ANKLE_LEFT', 'KNEE_RIGHT', 'KNEE_LEFT',
    'HIP_RIGHT', 'HIP_LEFT', 'PELVIS', 'SPINE_NAVAL', 'SPINE_CHEST',
    'CLAVICLE_RIGHT', 'CLAVICLE_LEFT', 'SHOULDER_RIGHT', 'SHOULDER_LEFT',
    'ELBOW_RIGHT', 'ELBOW_LEFT', 'WRIST_RIGHT', 'WRIST_LEFT', 'HAND_RIGHT',
    'HAND_LEFT', 'HANDTIP_RIGHT', 'HANDTIP_LEFT', 'THUMB_RIGHT', 'THUMB_LEFT',
    'NECK', 'HEAD'
]

# Create mock data for a humanoid figure
def create_mock_data():
    mock_data = {
        'FOOT_RIGHT_X': 0, 'FOOT_RIGHT_Y': 0, 'FOOT_RIGHT_Z': 0,
        'FOOT_LEFT_X': 1, 'FOOT_LEFT_Y': 0, 'FOOT_LEFT_Z': 0,
        'ANKLE_RIGHT_X': 0, 'ANKLE_RIGHT_Y': 1, 'ANKLE_RIGHT_Z': 0,
        'ANKLE_LEFT_X': 1, 'ANKLE_LEFT_Y': 1, 'ANKLE_LEFT_Z': 0,
        'KNEE_RIGHT_X': 0, 'KNEE_RIGHT_Y': 2, 'KNEE_RIGHT_Z': 0,
        'KNEE_LEFT_X': 1, 'KNEE_LEFT_Y': 2, 'KNEE_LEFT_Z': 0,
        'HIP_RIGHT_X': 0, 'HIP_RIGHT_Y': 3, 'HIP_RIGHT_Z': 0,
        'HIP_LEFT_X': 1, 'HIP_LEFT_Y': 3, 'HIP_LEFT_Z': 0,
        'PELVIS_X': 0.5, 'PELVIS_Y': 3.5, 'PELVIS_Z': 0,
        'SPINE_NAVAL_X': 0.5, 'SPINE_NAVAL_Y': 4, 'SPINE_NAVAL_Z': 0,
        'SPINE_CHEST_X': 0.5, 'SPINE_CHEST_Y': 5, 'SPINE_CHEST_Z': 0,
        'CLAVICLE_RIGHT_X': 0, 'CLAVICLE_RIGHT_Y': 5.5, 'CLAVICLE_RIGHT_Z': 0,
        'CLAVICLE_LEFT_X': 1, 'CLAVICLE_LEFT_Y': 5.5, 'CLAVICLE_LEFT_Z': 0,
        'SHOULDER_RIGHT_X': 0, 'SHOULDER_RIGHT_Y': 6, 'SHOULDER_RIGHT_Z': 0,
        'SHOULDER_LEFT_X': 1, 'SHOULDER_LEFT_Y': 6, 'SHOULDER_LEFT_Z': 0,
        'ELBOW_RIGHT_X': 0, 'ELBOW_RIGHT_Y': 7, 'ELBOW_RIGHT_Z': 0,
        'ELBOW_LEFT_X': 1, 'ELBOW_LEFT_Y': 7, 'ELBOW_LEFT_Z': 0,
        'WRIST_RIGHT_X': 0, 'WRIST_RIGHT_Y': 8, 'WRIST_RIGHT_Z': 0,
        'WRIST_LEFT_X': 1, 'WRIST_LEFT_Y': 8, 'WRIST_LEFT_Z': 0,
        'HAND_RIGHT_X': 0, 'HAND_RIGHT_Y': 9, 'HAND_RIGHT_Z': 0,
        'HAND_LEFT_X': 1, 'HAND_LEFT_Y': 9, 'HAND_LEFT_Z': 0,
        'HANDTIP_RIGHT_X': 0, 'HANDTIP_RIGHT_Y': 9.5, 'HANDTIP_RIGHT_Z': 0,
        'HANDTIP_LEFT_X': 1, 'HANDTIP_LEFT_Y': 9.5, 'HANDTIP_LEFT_Z': 0,
        'THUMB_RIGHT_X': -0.5, 'THUMB_RIGHT_Y': 9, 'THUMB_RIGHT_Z': 0,
        'THUMB_LEFT_X': 1.5, 'THUMB_LEFT_Y': 9, 'THUMB_LEFT_Z': 0,
        'NECK_X': 0.5, 'NECK_Y': 6.5, 'NECK_Z': 0,
        'HEAD_X': 0.5, 'HEAD_Y': 7, 'HEAD_Z': 0,
    }

    return pd.Series(mock_data)

# Function to extract node features and create a Data object
def create_data_object_vis(row):
    node_features = []
    node_positions = []  # Store (x, y, z) positions separately for Plotly visualization

    for joint in joints:
        x = row[f'{joint}_X']
        y = row[f'{joint}_Y']
        z = row[f'{joint}_Z']
        node_features.append([x, y, z])
        node_positions.append((x, y, z))  # Save position for each joint

    node_features = torch.tensor(node_features, dtype=torch.float)

    # No label in this mock data
    label = torch.tensor([0], dtype=torch.float)

    # Create the Data object
    data = Data(x=node_features, edge_index=edges, y=label)

    return data, node_positions

# Function to plot the graph using Plotly
def plot_graph_large(node_positions):
    # Extract x, y, z coordinates from node_positions
    x_coords = [pos[0] for pos in node_positions]
    y_coords = [pos[1] for pos in node_positions]
    z_coords = [pos[2] for pos in node_positions]

    # Create edges for Plotly
    edge_x = []
    edge_y = []
    edge_z = []
    for edge in edges.t().tolist():
        x0, y0, z0 = node_positions[edge[0]]
        x1, y1, z1 = node_positions[edge[1]]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])
        edge_z.extend([z0, z1, None])

    # Plot the edges
    edge_trace = go.Scatter3d(
        x=edge_x, y=edge_y, z=edge_z,
        mode='lines',
        line=dict(color='black', width=3),  # Increased width for better visibility
        hoverinfo='none'
    )

    # Plot the nodes with joint names
    node_trace = go.Scatter3d(
        x=x_coords, y=y_coords, z=z_coords,
        mode='markers+text',
        marker=dict(size=8, color='blue'),  # Increased size for better visibility
        text=joints,  # Use the joint names as text labels
        hoverinfo='text'
    )

    # Create the figure with increased size
    fig = go.Figure(data=[edge_trace, node_trace],
                    layout=go.Layout(
                        title='3D Humanoid Skeleton Visualization',
                        showlegend=False,
                        scene=dict(
                            xaxis=dict(
                                showbackground=True,
                                backgroundcolor="rgb(230, 230, 230)",
                                gridcolor="rgb(200, 200, 200)",
                                showgrid=True,
                                zerolinecolor="rgb(200, 200, 200)",
                            ),
                            yaxis=dict(
                                showbackground=True,
                                backgroundcolor="rgb(230, 230, 230)",
                                gridcolor="rgb(200, 200, 200)",
                                showgrid=True,
                                zerolinecolor="rgb(200, 200, 200)",
                            ),
                            zaxis=dict(
                                showbackground=True,
                                backgroundcolor="rgb(230, 230, 230)",
                                gridcolor="rgb(200, 200, 200)",
                                showgrid=True,
                                zerolinecolor="rgb(200, 200, 200)",
                            ),
                        ),
                        width=1200,  # Increased width
                        height=800,  # Increased height
                        margin=dict(l=0, r=0, b=0, t=40),
                    ))

    fig.show()

# Create mock data for a humanoid figure
mock_data = create_mock_data()

# Convert the mock data to a Data object and node positions
data, node_positions = create_data_object_vis(mock_data)

# Plot the larger graph using Plotly
plot_graph_large(node_positions)


In [20]:
import torch
import pandas as pd
import plotly.graph_objs as go
from torch_geometric.data import Data

# Load the dataset

df = pd.read_csv(file_path)

# Define anatomical connections as edges
edges = torch.tensor([
    [0, 2],  # FOOT_RIGHT to ANKLE_RIGHT
    [1, 3],  # FOOT_LEFT to ANKLE_LEFT
    [2, 4],  # ANKLE_RIGHT to KNEE_RIGHT
    [3, 5],  # ANKLE_LEFT to KNEE_LEFT
    [4, 6],  # KNEE_RIGHT to HIP_RIGHT
    [5, 7],  # KNEE_LEFT to HIP_LEFT
    [6, 8],  # HIP_RIGHT to PELVIS
    [7, 8],  # HIP_LEFT to PELVIS
    [8, 9],  # PELVIS to SPINE_NAVAL
    [9, 10], # SPINE_NAVAL to SPINE_CHEST
    [10, 11],# SPINE_CHEST to CLAVICLE_RIGHT
    [10, 12],# SPINE_CHEST to CLAVICLE_LEFT
    [11, 13],# CLAVICLE_RIGHT to SHOULDER_RIGHT
    [12, 14],# CLAVICLE_LEFT to SHOULDER_LEFT
    [13, 15],# SHOULDER_RIGHT to ELBOW_RIGHT
    [14, 16],# SHOULDER_LEFT to ELBOW_LEFT
    [15, 17],# ELBOW_RIGHT to WRIST_RIGHT
    [16, 18],# ELBOW_LEFT to WRIST_LEFT
    [17, 19],# WRIST_RIGHT to HAND_RIGHT
    [18, 20],# WRIST_LEFT to HAND_LEFT
    [19, 21],# HAND_RIGHT to HANDTIP_RIGHT
    [20, 22],# HAND_LEFT to HANDTIP_LEFT
    [21, 23],# HANDTIP_RIGHT to THUMB_RIGHT
    [22, 24],# HANDTIP_LEFT to THUMB_LEFT
    [10, 25],# SPINE_CHEST to NECK
    [25, 26],# NECK to HEAD
]).t().contiguous()

# List of joints in the order they appear in the DataFrame
joints = [
    'FOOT_RIGHT', 'FOOT_LEFT', 'ANKLE_RIGHT', 'ANKLE_LEFT', 'KNEE_RIGHT', 'KNEE_LEFT',
    'HIP_RIGHT', 'HIP_LEFT', 'PELVIS', 'SPINE_NAVAL', 'SPINE_CHEST',
    'CLAVICLE_RIGHT', 'CLAVICLE_LEFT', 'SHOULDER_RIGHT', 'SHOULDER_LEFT',
    'ELBOW_RIGHT', 'ELBOW_LEFT', 'WRIST_RIGHT', 'WRIST_LEFT', 'HAND_RIGHT',
    'HAND_LEFT', 'HANDTIP_RIGHT', 'HANDTIP_LEFT', 'THUMB_RIGHT', 'THUMB_LEFT',
    'NECK', 'HEAD', 'NOSE', 'EYE_LEFT', 'EAR_LEFT', 'EYE_RIGHT', 'EAR_RIGHT'
]

# Function to extract node features and create a Data object
def create_data_object_vis(row):
    node_features = []
    node_positions = []  # Store (x, y, z) positions separately for Plotly visualization

    for joint in joints:
        x = row[f'{joint}_X']
        y = row[f'{joint}_Y']
        z = row[f'{joint}_Z']
        node_features.append([x, y, z])
        node_positions.append((x, y, z))  # Save position for each joint

    node_features = torch.tensor(node_features, dtype=torch.float)

    # Extract label (assuming 'frailty' is the target column)
    label = torch.tensor([row['QoR_class']], dtype=torch.float)

    # Create the Data object
    data = Data(x=node_features, edge_index=edges, y=label)

    return data, node_positions

# Function to plot the animated graph using Plotly
def plot_graph_animation(df, start_row=0, end_row=10):
    frames = []

    for i in range(start_row, end_row + 1):
        single_row = df.iloc[i]
        _, node_positions = create_data_object_vis(single_row)

        x_coords = [pos[0] for pos in node_positions]
        y_coords = [pos[1] for pos in node_positions]
        z_coords = [pos[2] for pos in node_positions]

        # Create edges for Plotly
        edge_x = []
        edge_y = []
        edge_z = []
        for edge in edges.t().tolist():
            x0, y0, z0 = node_positions[edge[0]]
            x1, y1, z1 = node_positions[edge[1]]
            edge_x.extend([x0, x1, None])
            edge_y.extend([y0, y1, None])
            edge_z.extend([z0, z1, None])

        # Create the scatter and line traces
        edge_trace = go.Scatter3d(
            x=edge_x, y=edge_y, z=edge_z,
            mode='lines',
            line=dict(color='black', width=2),
            hoverinfo='none'
        )

        node_trace = go.Scatter3d(
            x=x_coords, y=y_coords, z=z_coords,
            mode='markers+text',
            marker=dict(size=6, color='blue'),
            text=joints,  # Use the joint names as text labels
            hoverinfo='text'
        )

        # Add the frame to the frames list
        frames.append(go.Frame(data=[edge_trace, node_trace], name=str(i)))

    # Create the figure with frames
    fig = go.Figure(
        data=[frames[0].data[0], frames[0].data[1]],
        layout=go.Layout(
            title='3D Joint Visualization Animation',
            showlegend=False,
            scene=dict(
                xaxis=dict(
                    showbackground=True,
                    backgroundcolor="rgb(230, 230, 230)",
                    gridcolor="rgb(200, 200, 200)",
                    showgrid=True,
                    zerolinecolor="rgb(200, 200, 200)",
                ),
                yaxis=dict(
                    showbackground=True,
                    backgroundcolor="rgb(230, 230, 230)",
                    gridcolor="rgb(200, 200, 200)",
                    showgrid=True,
                    zerolinecolor="rgb(200, 200, 200)",
                ),
                zaxis=dict(
                    showbackground=True,
                    backgroundcolor="rgb(230, 230, 230)",
                    gridcolor="rgb(200, 200, 200)",
                    showgrid=True,
                    zerolinecolor="rgb(200, 200, 200)",
                ),
            ),
            margin=dict(l=0, r=0, b=0, t=40),
            updatemenus=[dict(type="buttons", showactive=False,
                              buttons=[dict(label="Play",
                                            method="animate",
                                            args=[None, dict(frame=dict(duration=500, redraw=True),
                                                             fromcurrent=True)])])]),
        frames=frames
    )

    fig.show()

# Call the function to plot the animation
plot_graph_animation(df, start_row=0, end_row=100)


In [21]:
import plotly.graph_objs as go
import networkx as nx

# Function to create a graph from a single row
def create_graph_from_row(row):
    G = nx.Graph()

    # Define the joints and their coordinates (X, Y, Z)
    joints = {
        'FOOT_RIGHT': (row['FOOT_RIGHT_X'], row['FOOT_RIGHT_Y'], row['FOOT_RIGHT_Z']),
        'FOOT_LEFT': (row['FOOT_LEFT_X'], row['FOOT_LEFT_Y'], row['FOOT_LEFT_Z']),
        'ANKLE_RIGHT': (row['ANKLE_RIGHT_X'], row['ANKLE_RIGHT_Y'], row['ANKLE_RIGHT_Z']),
        'ANKLE_LEFT': (row['ANKLE_LEFT_X'], row['ANKLE_LEFT_Y'], row['ANKLE_LEFT_Z']),
        'KNEE_RIGHT': (row['KNEE_RIGHT_X'], row['KNEE_RIGHT_Y'], row['KNEE_RIGHT_Z']),
        'KNEE_LEFT': (row['KNEE_LEFT_X'], row['KNEE_LEFT_Y'], row['KNEE_LEFT_Z']),
        'HIP_RIGHT': (row['HIP_RIGHT_X'], row['HIP_RIGHT_Y'], row['HIP_RIGHT_Z']),
        'HIP_LEFT': (row['HIP_LEFT_X'], row['HIP_LEFT_Y'], row['HIP_LEFT_Z']),
        'PELVIS': (row['PELVIS_X'], row['PELVIS_Y'], row['PELVIS_Z']),
        'SPINE_NAVAL': (row['SPINE_NAVAL_X'], row['SPINE_NAVAL_Y'], row['SPINE_NAVAL_Z']),
        'SPINE_CHEST': (row['SPINE_CHEST_X'], row['SPINE_CHEST_Y'], row['SPINE_CHEST_Z']),
        'CLAVICLE_RIGHT': (row['CLAVICLE_RIGHT_X'], row['CLAVICLE_RIGHT_Y'], row['CLAVICLE_RIGHT_Z']),
        'CLAVICLE_LEFT': (row['CLAVICLE_LEFT_X'], row['CLAVICLE_LEFT_Y'], row['CLAVICLE_LEFT_Z']),
        'SHOULDER_RIGHT': (row['SHOULDER_RIGHT_X'], row['SHOULDER_RIGHT_Y'], row['SHOULDER_RIGHT_Z']),
        'SHOULDER_LEFT': (row['SHOULDER_LEFT_X'], row['SHOULDER_LEFT_Y'], row['SHOULDER_LEFT_Z']),
        'ELBOW_RIGHT': (row['ELBOW_RIGHT_X'], row['ELBOW_RIGHT_Y'], row['ELBOW_RIGHT_Z']),
        'ELBOW_LEFT': (row['ELBOW_LEFT_X'], row['ELBOW_LEFT_Y'], row['ELBOW_LEFT_Z']),
        'WRIST_RIGHT': (row['WRIST_RIGHT_X'], row['WRIST_RIGHT_Y'], row['WRIST_RIGHT_Z']),
        'WRIST_LEFT': (row['WRIST_LEFT_X'], row['WRIST_LEFT_Y'], row['WRIST_LEFT_Z']),
        'HAND_RIGHT': (row['HAND_RIGHT_X'], row['HAND_RIGHT_Y'], row['HAND_RIGHT_Z']),
        'HAND_LEFT': (row['HAND_LEFT_X'], row['HAND_LEFT_Y'], row['HAND_LEFT_Z']),
        'HANDTIP_RIGHT': (row['HANDTIP_RIGHT_X'], row['HANDTIP_RIGHT_Y'], row['HANDTIP_RIGHT_Z']),
        'HANDTIP_LEFT': (row['HANDTIP_LEFT_X'], row['HANDTIP_LEFT_Y'], row['HANDTIP_LEFT_Z']),
        'THUMB_RIGHT': (row['THUMB_RIGHT_X'], row['THUMB_RIGHT_Y'], row['THUMB_RIGHT_Z']),
        'THUMB_LEFT': (row['THUMB_LEFT_X'], row['THUMB_LEFT_Y'], row['THUMB_LEFT_Z']),
        'NECK': (row['NECK_X'], row['NECK_Y'], row['NECK_Z']),
        'HEAD': (row['HEAD_X'], row['HEAD_Y'], row['HEAD_Z']),
    }

    # Add nodes to the graph
    for joint, coords in joints.items():
        G.add_node(joint, pos=coords)

    # Define edges based on anatomical connections
    G.add_edges_from([
        # Legs
        ('FOOT_RIGHT', 'ANKLE_RIGHT'),
        ('FOOT_LEFT', 'ANKLE_LEFT'),
        ('ANKLE_RIGHT', 'KNEE_RIGHT'),
        ('ANKLE_LEFT', 'KNEE_LEFT'),
        ('KNEE_RIGHT', 'HIP_RIGHT'),
        ('KNEE_LEFT', 'HIP_LEFT'),
        ('HIP_RIGHT', 'PELVIS'),
        ('HIP_LEFT', 'PELVIS'),

        # Spine and arms
        ('PELVIS', 'SPINE_NAVAL'),
        ('SPINE_NAVAL', 'SPINE_CHEST'),
        ('SPINE_CHEST', 'CLAVICLE_RIGHT'),
        ('SPINE_CHEST', 'CLAVICLE_LEFT'),
        ('CLAVICLE_RIGHT', 'SHOULDER_RIGHT'),
        ('CLAVICLE_LEFT', 'SHOULDER_LEFT'),
        ('SHOULDER_RIGHT', 'ELBOW_RIGHT'),
        ('SHOULDER_LEFT', 'ELBOW_LEFT'),
        ('ELBOW_RIGHT', 'WRIST_RIGHT'),
        ('ELBOW_LEFT', 'WRIST_LEFT'),
        ('WRIST_RIGHT', 'HAND_RIGHT'),
        ('WRIST_LEFT', 'HAND_LEFT'),
        ('HAND_RIGHT', 'HANDTIP_RIGHT'),
        ('HAND_LEFT', 'HANDTIP_LEFT'),
        ('HANDTIP_RIGHT', 'THUMB_RIGHT'),
        ('HANDTIP_LEFT', 'THUMB_LEFT'),

        # Head and neck
        ('SPINE_CHEST', 'NECK'),
        ('NECK', 'HEAD'),
    ])

    return G

# Function to visualize the graph in 3D with Plotly
def visualize_graph_3d_plotly(G):
    pos = nx.get_node_attributes(G, 'pos')

    # Create edges
    edge_x = []
    edge_y = []
    edge_z = []
    for edge in G.edges():
        x0, y0, z0 = pos[edge[0]]
        x1, y1, z1 = pos[edge[1]]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])
        edge_z.extend([z0, z1, None])

    edge_trace = go.Scatter3d(
        x=edge_x, y=edge_y, z=edge_z,
        line=dict(width=2, color='black'),
        hoverinfo='none',
        mode='lines')

    # Create nodes
    node_x = []
    node_y = []
    node_z = []
    node_text = []
    for node in G.nodes():
        x, y, z = pos[node]
        node_x.append(x)
        node_y.append(y)
        node_z.append(z)
        node_text.append(node)

    node_trace = go.Scatter3d(
        x=node_x, y=node_y, z=node_z,
        mode='markers+text',
        marker=dict(
            size=8,
            color='blue',
            line=dict(width=2)),
        text=node_text,
        hoverinfo='text')

    # Create the figure with gridlines
    fig = go.Figure(data=[edge_trace, node_trace],
                    layout=go.Layout(
                        title='3D Human Skeleton Visualization with Grid',
                        showlegend=False,
                        scene=dict(
                            xaxis=dict(
                                showbackground=True,
                                backgroundcolor="rgb(230, 230, 230)",
                                gridcolor="rgb(200, 200, 200)",
                                showgrid=True,
                                zerolinecolor="rgb(200, 200, 200)",
                            ),
                            yaxis=dict(
                                showbackground=True,
                                backgroundcolor="rgb(230, 230, 230)",
                                gridcolor="rgb(200, 200, 200)",
                                showgrid=True,
                                zerolinecolor="rgb(200, 200, 200)",
                            ),
                            zaxis=dict(
                                showbackground=True,
                                backgroundcolor="rgb(230, 230, 230)",
                                gridcolor="rgb(200, 200, 200)",
                                showgrid=True,
                                zerolinecolor="rgb(200, 200, 200)",
                            ),
                        ),
                        margin=dict(l=0, r=0, b=0, t=40),
                    ))

    fig.show()

# Example usage
# Replace df.iloc[10] with an actual DataFrame row containing the skeleton data
G = create_graph_from_row(df.iloc[10])
visualize_graph_3d_plotly(G)


In [22]:
import torch
import pandas as pd
import plotly.graph_objs as go
from torch_geometric.data import Data

# Load the dataset

df = pd.read_csv(file_path)

# Define anatomical connections as edges
edges = torch.tensor([
    [0, 2],  # FOOT_RIGHT to ANKLE_RIGHT
    [1, 3],  # FOOT_LEFT to ANKLE_LEFT
    [2, 4],  # ANKLE_RIGHT to KNEE_RIGHT
    [3, 5],  # ANKLE_LEFT to KNEE_LEFT
    [4, 6],  # KNEE_RIGHT to HIP_RIGHT
    [5, 7],  # KNEE_LEFT to HIP_LEFT
    [6, 8],  # HIP_RIGHT to PELVIS
    [7, 8],  # HIP_LEFT to PELVIS
    [8, 9],  # PELVIS to SPINE_NAVAL
    [9, 10], # SPINE_NAVAL to SPINE_CHEST
    [10, 11],# SPINE_CHEST to CLAVICLE_RIGHT
    [10, 12],# SPINE_CHEST to CLAVICLE_LEFT
    [11, 13],# CLAVICLE_RIGHT to SHOULDER_RIGHT
    [12, 14],# CLAVICLE_LEFT to SHOULDER_LEFT
    [13, 15],# SHOULDER_RIGHT to ELBOW_RIGHT
    [14, 16],# SHOULDER_LEFT to ELBOW_LEFT
    [15, 17],# ELBOW_RIGHT to WRIST_RIGHT
    [16, 18],# ELBOW_LEFT to WRIST_LEFT
    [17, 19],# WRIST_RIGHT to HAND_RIGHT
    [18, 20],# WRIST_LEFT to HAND_LEFT
    [19, 21],# HAND_RIGHT to HANDTIP_RIGHT
    [20, 22],# HAND_LEFT to HANDTIP_LEFT
    [21, 23],# HANDTIP_RIGHT to THUMB_RIGHT
    [22, 24],# HANDTIP_LEFT to THUMB_LEFT
    [10, 25],# SPINE_CHEST to NECK
    [25, 26],# NECK to HEAD
]).t().contiguous()

# List of joints in the order they appear in the DataFrame
joints = [
    'FOOT_RIGHT', 'FOOT_LEFT', 'ANKLE_RIGHT', 'ANKLE_LEFT', 'KNEE_RIGHT', 'KNEE_LEFT',
    'HIP_RIGHT', 'HIP_LEFT', 'PELVIS', 'SPINE_NAVAL', 'SPINE_CHEST',
    'CLAVICLE_RIGHT', 'CLAVICLE_LEFT', 'SHOULDER_RIGHT', 'SHOULDER_LEFT',
    'ELBOW_RIGHT', 'ELBOW_LEFT', 'WRIST_RIGHT', 'WRIST_LEFT', 'HAND_RIGHT',
    'HAND_LEFT', 'HANDTIP_RIGHT', 'HANDTIP_LEFT', 'THUMB_RIGHT', 'THUMB_LEFT',
    'NECK', 'HEAD', 'NOSE', 'EYE_LEFT', 'EAR_LEFT', 'EYE_RIGHT', 'EAR_RIGHT'
]

# Function to extract node features and create a Data object
def create_data_object_vis(row):
    node_features = []
    node_positions = []  # Store (x, y, z) positions separately for Plotly visualization

    for joint in joints:
        x = row[f'{joint}_X']
        y = row[f'{joint}_Y']
        z = row[f'{joint}_Z']
        node_features.append([x, y, z])
        node_positions.append((x, y, z))  # Save position for each joint

    node_features = torch.tensor(node_features, dtype=torch.float)

    # Extract label (assuming 'frailty' is the target column)
    label = torch.tensor([row['QoR_class']], dtype=torch.float)

    # Create the Data object
    data = Data(x=node_features, edge_index=edges, y=label)

    return data, node_positions

# Function to plot the graph using Plotly
def plot_graph(node_positions):
    # Extract x, y, z coordinates from node_positions
    x_coords = [pos[0] for pos in node_positions]
    y_coords = [pos[1] for pos in node_positions]
    z_coords = [pos[2] for pos in node_positions]

    # Create edges for Plotly
    edge_x = []
    edge_y = []
    edge_z = []
    for edge in edges.t().tolist():
        x0, y0, z0 = node_positions[edge[0]]
        x1, y1, z1 = node_positions[edge[1]]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])
        edge_z.extend([z0, z1, None])

    # Plot the edges
    edge_trace = go.Scatter3d(
        x=edge_x, y=edge_y, z=edge_z,
        mode='lines',
        line=dict(color='black', width=2),
        hoverinfo='none'
    )

    # Plot the nodes with joint names
    node_trace = go.Scatter3d(
        x=x_coords, y=y_coords, z=z_coords,
        mode='markers+text',
        marker=dict(size=6, color='blue'),
        text=joints,  # Use the joint names as text labels
        hoverinfo='text'
    )

    # Create the figure with gridlines and joint names
    fig = go.Figure(data=[edge_trace, node_trace],
                    layout=go.Layout(
                        title='3D  Visualization',
                        showlegend=False,
                        scene=dict(
                            xaxis=dict(
                                showbackground=True,
                                backgroundcolor="rgb(230, 230, 230)",
                                gridcolor="rgb(200, 200, 200)",
                                showgrid=True,
                                zerolinecolor="rgb(200, 200, 200)",
                            ),
                            yaxis=dict(
                                showbackground=True,
                                backgroundcolor="rgb(230, 230, 230)",
                                gridcolor="rgb(200, 200, 200)",
                                showgrid=True,
                                zerolinecolor="rgb(200, 200, 200)",
                            ),
                            zaxis=dict(
                                showbackground=True,
                                backgroundcolor="rgb(230, 230, 230)",
                                gridcolor="rgb(200, 200, 200)",
                                showgrid=True,
                                zerolinecolor="rgb(200, 200, 200)",
                            ),
                        ),
                        margin=dict(l=0, r=0, b=0, t=40),
                    ))

    fig.show()

# Example usage with a single row (e.g., the first row in your DataFrame)
single_row = df.iloc[0]
data, node_positions = create_data_object_vis(single_row)

# Plot the graph using Plotly
plot_graph(node_positions)


In [26]:
import torch
import pandas as pd
import plotly.graph_objs as go
from torch_geometric.data import Data

# Load the dataset

df = pd.read_csv(file_path)

# Define anatomical connections as edges
edges = torch.tensor([
    [0, 2],  # FOOT_RIGHT to ANKLE_RIGHT
    [1, 3],  # FOOT_LEFT to ANKLE_LEFT
    [2, 4],  # ANKLE_RIGHT to KNEE_RIGHT
    [3, 5],  # ANKLE_LEFT to KNEE_LEFT
    [4, 6],  # KNEE_RIGHT to HIP_RIGHT
    [5, 7],  # KNEE_LEFT to HIP_LEFT
    [6, 8],  # HIP_RIGHT to PELVIS
    [7, 8],  # HIP_LEFT to PELVIS
    [8, 9],  # PELVIS to SPINE_NAVAL
    [9, 10], # SPINE_NAVAL to SPINE_CHEST
    [10, 11],# SPINE_CHEST to CLAVICLE_RIGHT
    [10, 12],# SPINE_CHEST to CLAVICLE_LEFT
    [11, 13],# CLAVICLE_RIGHT to SHOULDER_RIGHT
    [12, 14],# CLAVICLE_LEFT to SHOULDER_LEFT
    [13, 15],# SHOULDER_RIGHT to ELBOW_RIGHT
    [14, 16],# SHOULDER_LEFT to ELBOW_LEFT
    [15, 17],# ELBOW_RIGHT to WRIST_RIGHT
    [16, 18],# ELBOW_LEFT to WRIST_LEFT
    [17, 19],# WRIST_RIGHT to HAND_RIGHT
    [18, 20],# WRIST_LEFT to HAND_LEFT
    [19, 21],# HAND_RIGHT to HANDTIP_RIGHT
    [20, 22],# HAND_LEFT to HANDTIP_LEFT
    [21, 23],# HANDTIP_RIGHT to THUMB_RIGHT
    [22, 24],# HANDTIP_LEFT to THUMB_LEFT
    [10, 25],# SPINE_CHEST to NECK
    [25, 26],# NECK to HEAD
]).t().contiguous()

# List of joints in the order they appear in the DataFrame
joints = [
    'FOOT_RIGHT', 'FOOT_LEFT', 'ANKLE_RIGHT', 'ANKLE_LEFT', 'KNEE_RIGHT', 'KNEE_LEFT',
    'HIP_RIGHT', 'HIP_LEFT', 'PELVIS', 'SPINE_NAVAL', 'SPINE_CHEST',
    'CLAVICLE_RIGHT', 'CLAVICLE_LEFT', 'SHOULDER_RIGHT', 'SHOULDER_LEFT',
    'ELBOW_RIGHT', 'ELBOW_LEFT', 'WRIST_RIGHT', 'WRIST_LEFT', 'HAND_RIGHT',
    'HAND_LEFT', 'HANDTIP_RIGHT', 'HANDTIP_LEFT', 'THUMB_RIGHT', 'THUMB_LEFT',
    'NECK', 'HEAD', 'NOSE', 'EYE_LEFT', 'EAR_LEFT', 'EYE_RIGHT', 'EAR_RIGHT'
]

# Function to extract node features and create a Data object
def create_data_object_vis(row):
    node_features = []
    node_positions = []  # Store (x, y, z) positions separately for Plotly visualization

    for joint in joints:
        x = row[f'{joint}_X']
        y = row[f'{joint}_Y']
        z = row[f'{joint}_Z']
        node_features.append([x, y, z])
        node_positions.append((x, y, z))  # Save position for each joint

    node_features = torch.tensor(node_features, dtype=torch.float)

    # Extract label (assuming 'frailty' is the target column)
    label = torch.tensor([row['QoR_class']], dtype=torch.float)

    # Create the Data object
    data = Data(x=node_features, edge_index=edges, y=label)

    return data, node_positions

# Function to plot the graph using Plotly
def plot_graph(node_positions):
    # Extract x, y, z coordinates from node_positions
    x_coords = [pos[0] for pos in node_positions]
    y_coords = [pos[1] for pos in node_positions]
    z_coords = [pos[2] for pos in node_positions]

    # Create edges for Plotly
    edge_x = []
    edge_y = []
    edge_z = []
    for edge in edges.t().tolist():
        x0, y0, z0 = node_positions[edge[0]]
        x1, y1, z1 = node_positions[edge[1]]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])
        edge_z.extend([z0, z1, None])

    # Plot the edges
    edge_trace = go.Scatter3d(
        x=edge_x, y=edge_y, z=edge_z,
        mode='lines',
        line=dict(color='black', width=2),
        hoverinfo='none'
    )

    # Plot the nodes with joint names
    node_trace = go.Scatter3d(
        x=x_coords, y=y_coords, z=z_coords,
        mode='markers+text',
        marker=dict(size=6, color='blue'),
        text=joints,  # Use the joint names as text labels
        hoverinfo='text'
    )

    # Create the figure with gridlines and joint names
    fig = go.Figure(data=[edge_trace, node_trace],
                    layout=go.Layout(
                        title='3D  Visualization',
                        showlegend=False,
                        scene=dict(
                            xaxis=dict(
                                showbackground=True,
                                backgroundcolor="rgb(230, 230, 230)",
                                gridcolor="rgb(200, 200, 200)",
                                showgrid=True,
                                zerolinecolor="rgb(200, 200, 200)",
                            ),
                            yaxis=dict(
                                showbackground=True,
                                backgroundcolor="rgb(230, 230, 230)",
                                gridcolor="rgb(200, 200, 200)",
                                showgrid=True,
                                zerolinecolor="rgb(200, 200, 200)",
                            ),
                            zaxis=dict(
                                showbackground=True,
                                backgroundcolor="rgb(230, 230, 230)",
                                gridcolor="rgb(200, 200, 200)",
                                showgrid=True,
                                zerolinecolor="rgb(200, 200, 200)",
                            ),
                        ),
                        margin=dict(l=0, r=0, b=0, t=40),
                    ))

    fig.show()

# Example usage with a single row (e.g., the first row in your DataFrame)
single_row = df.iloc[0]
data, node_positions = create_data_object_vis(single_row)

# Plot the graph using Plotly
plot_graph(node_positions)


In [ ]:
# Define anatomical connections as edges
edges = torch.tensor([
    [0, 2],  # FOOT_RIGHT to ANKLE_RIGHT
    [1, 3],  # FOOT_LEFT to ANKLE_LEFT
    [2, 4],  # ANKLE_RIGHT to KNEE_RIGHT
    [3, 5],  # ANKLE_LEFT to KNEE_LEFT
    [4, 6],  # KNEE_RIGHT to HIP_RIGHT
    [5, 7],  # KNEE_LEFT to HIP_LEFT
    [6, 8],  # HIP_RIGHT to PELVIS
    [7, 8],  # HIP_LEFT to PELVIS
    [8, 9],  # PELVIS to SPINE_NAVAL
    [9, 10], # SPINE_NAVAL to SPINE_CHEST
    [10, 11],# SPINE_CHEST to CLAVICLE_RIGHT
    [10, 12],# SPINE_CHEST to CLAVICLE_LEFT
    [11, 13],# CLAVICLE_RIGHT to SHOULDER_RIGHT
    [12, 14],# CLAVICLE_LEFT to SHOULDER_LEFT
    [13, 15],# SHOULDER_RIGHT to ELBOW_RIGHT
    [14, 16],# SHOULDER_LEFT to ELBOW_LEFT
    [15, 17],# ELBOW_RIGHT to WRIST_RIGHT
    [16, 18],# ELBOW_LEFT to WRIST_LEFT
    [17, 19],# WRIST_RIGHT to HAND_RIGHT
    [18, 20],# WRIST_LEFT to HAND_LEFT
    [19, 21],# HAND_RIGHT to HANDTIP_RIGHT
    [20, 22],# HAND_LEFT to HANDTIP_LEFT
    [21, 23],# HANDTIP_RIGHT to THUMB_RIGHT
    [22, 24],# HANDTIP_LEFT to THUMB_LEFT
    [10, 25],# SPINE_CHEST to NECK
    [25, 26],# NECK to HEAD
]).t().contiguous()

# Function to extract node features and create a Data object
def create_data_object(row):
    node_features = []

    for joint in ['FOOT_RIGHT', 'FOOT_LEFT', 'ANKLE_RIGHT', 'ANKLE_LEFT', 'KNEE_RIGHT', 'KNEE_LEFT',
                  'HIP_RIGHT', 'HIP_LEFT', 'PELVIS', 'SPINE_NAVAL', 'SPINE_CHEST',
                  'CLAVICLE_RIGHT', 'CLAVICLE_LEFT', 'SHOULDER_RIGHT', 'SHOULDER_LEFT',
                  'ELBOW_RIGHT', 'ELBOW_LEFT', 'WRIST_RIGHT', 'WRIST_LEFT', 'HAND_RIGHT',
                  'HAND_LEFT', 'HANDTIP_RIGHT', 'HANDTIP_LEFT', 'THUMB_RIGHT', 'THUMB_LEFT',
                  'NECK', 'HEAD', 'NOSE', 'EYE_LEFT', 'EAR_LEFT', 'EYE_RIGHT', 'EAR_RIGHT']:

        x = row[f'{joint}_X']
        y = row[f'{joint}_Y']
        z = row[f'{joint}_Z']
        node_features.append([x, y, z])

    node_features = torch.tensor(node_features, dtype=torch.float)

    # Extract label (assuming 'frailty' is the target column)
    label = torch.tensor([row['frailty_class']], dtype=torch.float)

    # Create the Data object
    data = Data(x=node_features, edge_index=edges, y=label)

    return data




# Apply the function to each row in the dataframe
data_list = [create_data_object(row) for index, row in df.iterrows()]

# Now data_list contains the graph data for each row in the dataset





In [28]:
import pandas as pd
import numpy as np
from glob import glob
from tqdm.notebook import tqdm
import torch
import plotly.graph_objs as go
from torch_geometric.data import Data
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import DataLoader, Dataset, Data
from torch_geometric.nn import GCNConv
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, roc_auc_score, confusion_matrix, roc_curve


# Define joints and edges
joints = [
    'PELVIS', 'SPINE_NAVAL', 'SPINE_CHEST', 'NECK', 'CLAVICLE_LEFT', 'SHOULDER_LEFT',
    'ELBOW_LEFT', 'WRIST_LEFT', 'HAND_LEFT', 'HANDTIP_LEFT', 'THUMB_LEFT',
    'CLAVICLE_RIGHT', 'SHOULDER_RIGHT', 'ELBOW_RIGHT', 'WRIST_RIGHT', 'HAND_RIGHT',
    'HANDTIP_RIGHT', 'THUMB_RIGHT', 'HIP_LEFT', 'KNEE_LEFT', 'ANKLE_LEFT',
    'FOOT_LEFT', 'HIP_RIGHT', 'KNEE_RIGHT', 'ANKLE_RIGHT', 'FOOT_RIGHT',
    'HEAD', 'NOSE', 'EYE_LEFT', 'EAR_LEFT', 'EYE_RIGHT', 'EAR_RIGHT'
]

edges = [
    ('PELVIS', 'SPINE_NAVAL'), ('SPINE_NAVAL', 'SPINE_CHEST'), ('SPINE_CHEST', 'NECK'),
    ('NECK', 'HEAD'), ('SPINE_CHEST', 'CLAVICLE_LEFT'), ('CLAVICLE_LEFT', 'SHOULDER_LEFT'),
    ('SHOULDER_LEFT', 'ELBOW_LEFT'), ('ELBOW_LEFT', 'WRIST_LEFT'), ('WRIST_LEFT', 'HAND_LEFT'),
    ('HAND_LEFT', 'HANDTIP_LEFT'), ('WRIST_LEFT', 'THUMB_LEFT'), ('SPINE_CHEST', 'CLAVICLE_RIGHT'),
    ('CLAVICLE_RIGHT', 'SHOULDER_RIGHT'), ('SHOULDER_RIGHT', 'ELBOW_RIGHT'), ('ELBOW_RIGHT', 'WRIST_RIGHT'),
    ('WRIST_RIGHT', 'HAND_RIGHT'), ('HAND_RIGHT', 'HANDTIP_RIGHT'), ('WRIST_RIGHT', 'THUMB_RIGHT'),
    ('PELVIS', 'HIP_LEFT'), ('HIP_LEFT', 'KNEE_LEFT'), ('KNEE_LEFT', 'ANKLE_LEFT'),
    ('ANKLE_LEFT', 'FOOT_LEFT'), ('PELVIS', 'HIP_RIGHT'), ('HIP_RIGHT', 'KNEE_RIGHT'),
    ('KNEE_RIGHT', 'ANKLE_RIGHT'), ('ANKLE_RIGHT', 'FOOT_RIGHT'),
    ('HEAD', 'NOSE'),('HEAD','EYE_LEFT'), ('HEAD', 'EYE_RIGHT'), ('HEAD', 'EAR_LEFT'), ('HEAD', 'EAR_RIGHT')
]

joint_to_idx = {joint: idx for idx, joint in enumerate(joints)}

edge_index = torch.tensor(
    [[joint_to_idx[src], joint_to_idx[dst]] for src, dst in edges] +
    [[joint_to_idx[dst], joint_to_idx[src]] for src, dst in edges],  # Bidirectional edges
    dtype=torch.long
).t()


# Custom Dataset Class
class SkeletonDataset(Dataset):
    def __init__(self, csv_file):
        self.data = csv_file
        self.num_nodes = len(joints)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]

        # Extract node features including timestamp, accel_energy_total, footfall_event_times
        node_features = []
        for joint in joints:
            x = row[f'{joint}_X']
            y = row[f'{joint}_Y']
            z = row[f'{joint}_Z']
            timestamp = row['t_uniform']  # Include timestamp
            accel_energy_total = row['accel_energy_total']  # Include accel_energy_total
            footfall_event_times = row['footfall_event_times']  # Include footfall_event_times
            node_features.append([x, y, z, timestamp, accel_energy_total, footfall_event_times])

        x = torch.tensor(node_features, dtype=torch.float)  # Shape: [num_nodes, 6]

        # Label
        y = torch.tensor(row['QoR_class'], dtype=torch.float)

        # Create data object
        data = Data(x=x, edge_index=edge_index, y=y)

        return data


class HybridSTGCN(nn.Module):
    def __init__(self, in_channels, hidden_channels, num_classes):
        super(HybridSTGCN, self).__init__()

        # Spatial Graph Convolution Layers
        self.gcn1 = GCNConv(in_channels, hidden_channels)
        self.gcn2 = GCNConv(hidden_channels, hidden_channels)

        # Temporal Convolution Layers
        self.temporal_conv1 = nn.Conv1d(in_channels=hidden_channels,
                                        out_channels=hidden_channels,
                                        kernel_size=3, padding=1)

        # LSTM for temporal modeling
        self.lstm = nn.LSTM(hidden_channels, hidden_channels, batch_first=True)

        # Fully Connected Layer
        self.fc = nn.Linear(hidden_channels, num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        batch = getattr(data, 'batch', None)  # Use batch if available, else None

        # Spatial GCN Layers
        x = F.relu(self.gcn1(x, edge_index))
        x = F.relu(self.gcn2(x, edge_index))  # x: [num_nodes*batch_size, hidden_channels]

        # Reshape for Temporal Conv
        if batch is not None:
            batch_size = batch.max().item() + 1
            num_nodes = x.size(0) // batch_size
        else:
            batch_size = 1
            num_nodes = x.size(0)

        x = x.view(batch_size, num_nodes, -1)  # [batch_size, num_nodes, hidden_channels]
        x = x.permute(0, 2, 1)  # [batch_size, hidden_channels, num_nodes]

        # Temporal Conv Layer
        x = F.relu(self.temporal_conv1(x))  # [batch_size, hidden_channels, num_nodes]

        # LSTM for capturing temporal patterns
        x = x.permute(0, 2, 1)  # [batch_size, num_nodes, hidden_channels]
        x, _ = self.lstm(x)  # [batch_size, num_nodes, hidden_channels]

        # Pooling over nodes
        x = x.mean(dim=1)  # [batch_size, hidden_channels]

        # Fully Connected Layer
        x = self.fc(x)  # [batch_size, num_classes]
        x = torch.sigmoid(x)  # For binary classification

        return x.squeeze()  # [batch_size]


# Data preparation

dataframe = pd.read_csv(file_path)
dataframe = dataframe[dataframe['walking_speed'] == "Fast"]

# Load dataset
dataset = SkeletonDataset(csv_file=dataframe[dataframe['walking_speed'] == "Fast"])

# Split into training and testing datasets
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# Create DataLoaders
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Initialize Model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
model = HybridSTGCN(in_channels=6, hidden_channels=128, num_classes=1).to(device)  # Adjust in_channels to 6

# Define Loss Function and Optimizer
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)



# Training Function
def train(epoch):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data)  # [batch_size]
        y = data.y.to(device).float()  # [batch_size]

        loss = criterion(out, y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * data.num_graphs
        preds = (out >= 0.5).float()
        correct += (preds == y).sum().item()
        total += data.num_graphs

    avg_loss = total_loss / len(train_loader.dataset)
    accuracy = correct / total
    print(f'Epoch {epoch}, Train Loss: {avg_loss:.4f}, Train Accuracy: {accuracy:.4f}')
    return avg_loss, accuracy




# Evaluation Function
def evaluate(loader):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for data in loader:
            data = data.to(device)
            out = model(data).view(-1)  # Ensure the output is of shape [batch_size]
            y = data.y.to(device).float().view(-1)  # Ensure target is of shape [batch_size]
            preds = (out >= 0.5).float()

            total_loss += criterion(out, y.float()).item() * data.num_graphs
            correct += (preds == y).sum().item()
            total += data.num_graphs
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(y.cpu().numpy())

    avg_loss = total_loss / len(loader.dataset)
    accuracy = correct / total
    return avg_loss, accuracy, all_preds, all_labels










class EarlyStopping:
    def __init__(self, patience=5, min_delta=0, restore_best_weights=True):
        self.patience = patience
        self.min_delta = min_delta
        self.restore_best_weights = restore_best_weights
        self.best_loss = float('inf')
        self.counter = 0
        self.best_model = None

    def __call__(self, val_loss, model):
        if val_loss < self.best_loss - self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
            if self.restore_best_weights:
                self.best_model = model.state_dict()
        else:
            self.counter += 1
            if self.counter >= self.patience:
                if self.restore_best_weights and self.best_model is not None:
                    model.load_state_dict(self.best_model)
                return True
        return False






# Initialize EarlyStopping
early_stopping = EarlyStopping(patience=3, min_delta=0.01, restore_best_weights=True)

# Training Loop with Early Stopping
num_epochs = 100
train_losses = []
test_losses = []
train_accuracies = []
test_accuracies = []

for epoch in range(1, num_epochs + 1):
    # Train the model and get training loss and accuracy
    train_loss, train_acc = train(epoch)
    train_losses.append(train_loss)
    train_accuracies.append(train_acc)

    # Evaluate the model on the test set and get test loss and accuracy
    test_loss, test_acc, all_preds, all_labels = evaluate(test_loader)
    test_losses.append(test_loss)
    test_accuracies.append(test_acc)

    print(f'Epoch {epoch}, Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}')

    # Check for early stopping
    if early_stopping(test_loss, model):
        print("Early stopping triggered.")
        break

# Final Evaluation Metrics
accuracy = accuracy_score(all_labels, all_preds)
f1 = f1_score(all_labels, all_preds)
recall = recall_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds)
roc_auc = roc_auc_score(all_labels, all_preds)

# Confusion Matrix
tn, fp, fn, tp = confusion_matrix(all_labels, all_preds).ravel()
specificity = tn / (tn + fp)

print(f'Final Test Accuracy: {accuracy:.4f}')
print(f'F1 Score: {f1:.4f}')
print(f'Recall: {recall:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Specificity: {specificity:.4f}')
print(f'AUC-ROC: {roc_auc:.4f}')









c:\Users\user\miniconda3\envs\torch_on\lib\site-packages\torch_geometric\deprecation.py:26: UserWarning:

'data.DataLoader' is deprecated, use 'loader.DataLoader' instead



cuda


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [34]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import DataLoader, Dataset, Data
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, roc_auc_score, confusion_matrix, roc_curve
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"







# Define joints and edges
joints = [
    'PELVIS', 'SPINE_NAVAL', 'SPINE_CHEST', 'NECK', 'CLAVICLE_LEFT', 'SHOULDER_LEFT',
    'ELBOW_LEFT', 'WRIST_LEFT', 'HAND_LEFT', 'HANDTIP_LEFT', 'THUMB_LEFT',
    'CLAVICLE_RIGHT', 'SHOULDER_RIGHT', 'ELBOW_RIGHT', 'WRIST_RIGHT', 'HAND_RIGHT',
    'HANDTIP_RIGHT', 'THUMB_RIGHT', 'HIP_LEFT', 'KNEE_LEFT', 'ANKLE_LEFT',
    'FOOT_LEFT', 'HIP_RIGHT', 'KNEE_RIGHT', 'ANKLE_RIGHT', 'FOOT_RIGHT',
    'HEAD', 'NOSE', 'EYE_LEFT', 'EAR_LEFT', 'EYE_RIGHT', 'EAR_RIGHT'
]

edges = [
    ('PELVIS', 'SPINE_NAVAL'), ('SPINE_NAVAL', 'SPINE_CHEST'), ('SPINE_CHEST', 'NECK'),
    ('NECK', 'HEAD'), ('SPINE_CHEST', 'CLAVICLE_LEFT'), ('CLAVICLE_LEFT', 'SHOULDER_LEFT'),
    ('SHOULDER_LEFT', 'ELBOW_LEFT'), ('ELBOW_LEFT', 'WRIST_LEFT'), ('WRIST_LEFT', 'HAND_LEFT'),
    ('HAND_LEFT', 'HANDTIP_LEFT'), ('WRIST_LEFT', 'THUMB_LEFT'), ('SPINE_CHEST', 'CLAVICLE_RIGHT'),
    ('CLAVICLE_RIGHT', 'SHOULDER_RIGHT'), ('SHOULDER_RIGHT', 'ELBOW_RIGHT'), ('ELBOW_RIGHT', 'WRIST_RIGHT'),
    ('WRIST_RIGHT', 'HAND_RIGHT'), ('HAND_RIGHT', 'HANDTIP_RIGHT'), ('WRIST_RIGHT', 'THUMB_RIGHT'),
    ('PELVIS', 'HIP_LEFT'), ('HIP_LEFT', 'KNEE_LEFT'), ('KNEE_LEFT', 'ANKLE_LEFT'),
    ('ANKLE_LEFT', 'FOOT_LEFT'), ('PELVIS', 'HIP_RIGHT'), ('HIP_RIGHT', 'KNEE_RIGHT'),
    ('KNEE_RIGHT', 'ANKLE_RIGHT'), ('ANKLE_RIGHT', 'FOOT_RIGHT'),
    ('HEAD', 'NOSE'),('HEAD','EYE_LEFT'), ('HEAD', 'EYE_RIGHT'), ('HEAD', 'EAR_LEFT'), ('HEAD', 'EAR_RIGHT')
]

joint_to_idx = {joint: idx for idx, joint in enumerate(joints)}

edge_index = torch.tensor(
    [[joint_to_idx[src], joint_to_idx[dst]] for src, dst in edges] +
    [[joint_to_idx[dst], joint_to_idx[src]] for src, dst in edges],  # Bidirectional edges
    dtype=torch.long
).t()


# Custom Dataset Class
class SkeletonDataset(Dataset):
    def __init__(self, csv_file):
        self.data = csv_file
        self.num_nodes = len(joints)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]

        # Extract node features and include timestamp
        node_features = []
        for joint in joints:
            x = row[f'{joint}_X']
            y = row[f'{joint}_Y']
            z = row[f'{joint}_Z']
            timestamp = row['timeStamps']  # Include timestamp as a feature
            node_features.append([x, y, z, timestamp])

        x = torch.tensor(node_features, dtype=torch.float)  # Shape: [num_nodes, 4]

        # Label
        y = torch.tensor(row['QoR_class'], dtype=torch.float)

        # Create data object
        data = Data(x=x, edge_index=edge_index, y=y)

        return data




class HybridSTGCN(nn.Module):
    def __init__(self, in_channels, hidden_channels, num_classes):
        super(HybridSTGCN, self).__init__()

        # Spatial Graph Convolution Layers
        self.gcn1 = GCNConv(in_channels, hidden_channels)
        self.gcn2 = GCNConv(hidden_channels, hidden_channels)

        # Temporal Convolution Layers
        self.temporal_conv1 = nn.Conv1d(in_channels=hidden_channels,
                                        out_channels=hidden_channels,
                                        kernel_size=3, padding=1)

        # LSTM for temporal modeling
        self.lstm = nn.LSTM(hidden_channels, hidden_channels, batch_first=True)

        # Fully Connected Layer
        self.fc = nn.Linear(hidden_channels, num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        print("Forward pass - x shape:", x.shape)
        print("Forward pass - edge_index shape:", edge_index.shape)
        batch = getattr(data, 'batch', None)  # Use batch if available, else None

        # Spatial GCN Layers
        x = F.relu(self.gcn1(x, edge_index))
        x = F.relu(self.gcn2(x, edge_index))  # x: [num_nodes*batch_size, hidden_channels]

        # Reshape for Temporal Conv
        if batch is not None:
            batch_size = batch.max().item() + 1
            num_nodes = x.size(0) // batch_size
        else:
            batch_size = 1
            num_nodes = x.size(0)

        x = x.view(batch_size, num_nodes, -1)  # [batch_size, num_nodes, hidden_channels]
        x = x.permute(0, 2, 1)  # [batch_size, hidden_channels, num_nodes]

        # Temporal Conv Layer
        x = F.relu(self.temporal_conv1(x))  # [batch_size, hidden_channels, num_nodes]

        # LSTM for capturing temporal patterns
        x = x.permute(0, 2, 1)  # [batch_size, num_nodes, hidden_channels]
        x, _ = self.lstm(x)  # [batch_size, num_nodes, hidden_channels]

        # Pooling over nodes
        x = x.mean(dim=1)  # [batch_size, hidden_channels]

        # Fully Connected Layer
        x = self.fc(x)  # [batch_size, num_classes]
        x = torch.sigmoid(x)  # For binary classification

        return x.squeeze()  # [batch_size]








dataframe = pd.read_csv(file_path)
dataframe = dataframe[dataframe['walking_speed'] == "Fast"]
# Step 1: Keep only valid labels (0 or 1)
dataframe = dataframe[dataframe['QoR_class'].isin([0.0, 1.0])].copy()

# Optional: Also drop any remaining NaNs (paranoia check)
dataframe = dataframe.dropna(subset=['QoR_class'])


# Load dataset
dataset = SkeletonDataset(csv_file=dataframe[dataframe['walking_speed'] == "Fast"])




# Split into training and testing datasets
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])



# Create DataLoaders
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)



# Initialize Model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = HybridSTGCN(in_channels=4, hidden_channels=128, num_classes=1).to(device)

# Define Loss Function and Optimizer
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)



# Training Function
def train(epoch):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data)  # [batch_size]
        y = data.y.to(device).float()  # [batch_size]

        loss = criterion(out, y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * data.num_graphs
        preds = (out >= 0.5).float()
        correct += (preds == y).sum().item()
        total += data.num_graphs

    avg_loss = total_loss / len(train_loader.dataset)
    accuracy = correct / total
    print(f'Epoch {epoch}, Train Loss: {avg_loss:.4f}, Train Accuracy: {accuracy:.4f}')
    return avg_loss, accuracy



# Evaluation Function
def evaluate(loader):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for data in loader:
            data = data.to(device)
            out = model(data).view(-1)  # Ensure the output is of shape [batch_size]
            y = data.y.to(device).float().view(-1)  # Ensure target is of shape [batch_size]
            preds = (out >= 0.5).float()

            total_loss += criterion(out, y.float()).item() * data.num_graphs
            correct += (preds == y).sum().item()
            total += data.num_graphs
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(y.cpu().numpy())

    avg_loss = total_loss / len(loader.dataset)
    accuracy = correct / total
    return avg_loss, accuracy, all_preds, all_labels


# Training Loop
num_epochs = 10
train_losses = []
test_losses = []
train_accuracies = []
test_accuracies = []

for epoch in range(1, num_epochs + 1):
    # Train the model and get training loss and accuracy
    train_loss, train_acc = train(epoch)
    train_losses.append(train_loss)
    train_accuracies.append(train_acc)

    # Evaluate the model on the test set and get test loss and accuracy
    test_loss, test_acc, all_preds, all_labels = evaluate(test_loader)
    test_losses.append(test_loss)
    test_accuracies.append(test_acc)

    print(f'Epoch {epoch}, Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}')

# Final Evaluation Metrics
accuracy = accuracy_score(all_labels, all_preds)
f1 = f1_score(all_labels, all_preds)
recall = recall_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds)
roc_auc = roc_auc_score(all_labels, all_preds)

# Confusion Matrix
tn, fp, fn, tp = confusion_matrix(all_labels, all_preds).ravel()
specificity = tn / (tn + fp)

print(f'Final Test Accuracy: {accuracy:.4f}')
print(f'F1 Score: {f1:.4f}')
print(f'Recall: {recall:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Specificity: {specificity:.4f}')
print(f'AUC-ROC: {roc_auc:.4f}')

#the accuracy can be boost to 90%+

c:\Users\user\miniconda3\envs\torch_on\lib\site-packages\torch_geometric\deprecation.py:26: UserWarning:

'data.DataLoader' is deprecated, use 'loader.DataLoader' instead



RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
print(dataframe['QoR_class'].unique())

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
